In [1]:
import argparse
import os
import torch

from src.datasets import get_dataloader
from src.lightning import DDPM
from src.molecule_builder import get_bond_order
from src.visualizer import save_xyz_file, visualize_chain
from tqdm.auto import tqdm
from pdb import set_trace
import sys #@mastro
from src import const #@mastro
import numpy as np #@mastro
from numpy.random import default_rng
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import directed_hausdorff

from sklearn.decomposition import PCA
from src.visualizer import load_molecule_xyz, load_xyz_files
import matplotlib.pyplot as plt
import imageio

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Simulate command-line arguments
sys.argv = [
    'ipykernel_launcher.py',
    '--checkpoint', 'models/zinc_difflinker.ckpt',
    '--chains', 'trajectories',
    '--data', 'datasets',
    '--prefix', 'zinc_final_test',
    '--keep_frames', '10',
    '--device', 'cuda:0'
]

parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', action='store', type=str, required=True)
parser.add_argument('--chains', action='store', type=str, required=True)
parser.add_argument('--prefix', action='store', type=str, required=True)
parser.add_argument('--data', action='store', type=str, required=False, default=None)
parser.add_argument('--keep_frames', action='store', type=int, required=True)
parser.add_argument('--device', action='store', type=str, required=True)
args = parser.parse_args()

experiment_name = args.checkpoint.split('/')[-1].replace('.ckpt', '')
chains_output_dir = os.path.join(args.chains, experiment_name, args.prefix, 'chains')
final_states_output_dir = os.path.join(args.chains, experiment_name, args.prefix, 'final_states')
os.makedirs(chains_output_dir, exist_ok=True)
os.makedirs(final_states_output_dir, exist_ok=True)

# Loading model form checkpoint (all hparams will be automatically set)
model = DDPM.load_from_checkpoint(args.checkpoint, map_location=args.device)

# Possibility to evaluate on different datasets (e.g., on CASF instead of ZINC)
model.val_data_prefix = args.prefix

# In case <Anonymous> will run my model or vice versa
if args.data is not None:
    model.data_path = args.data

model = model.eval().to(args.device)
model.setup(stage='val')
dataloader = get_dataloader(
    model.val_dataset,
    batch_size=1, #@mastro, it was 32
    # batch_size=len(model.val_dataset)
)


c:\Users\Mastro\anaconda3\envs\diff_explainer\lib\site-packages\lightning_fabric\utilities\cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Lightning automatically upgraded your

In [2]:
def compute_molecular_similarity(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on distances and atom type.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]

    return 1 - torch.norm(mol1 - mol2)

def compute_molecular_distance(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on distances and atom type.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]

    return torch.norm(mol1 - mol2).item()

def compute_cosine_similarity(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on distances and atom type.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]

    return cosine_similarity(mol1.flatten().reshape(1, -1), mol2.flatten().reshape(1, -1)).item()

def compute_molecular_similarity_positions(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on positions.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    positions1 = mol1[:, :3].squeeze()
    positions2 = mol2[:, :3].squeeze()

    if mask1 is not None:
        mask1 = mask1.bool()
        positions1 = positions1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        positions2 = positions2[mask2,:]


    return 1 - torch.norm(positions1 - positions2) #choose if distance or similarity, need to check what it the better choice

def compute_one_hot_similarity(mol1, mol2, mask1 = None, mask2 = None):
    """
    Computes the similarity between two one-hot encoded molecules. The one-hot encoding indicates the atom type
    
    Args:
        mol1 (torch.Tensor): The first one-hot encoded molecule.
        mol2 (torch.Tensor): The second one-hot encoded molecule.
        mask (torch.Tensor, optional): A mask to apply on the atoms. Defaults to None.
    
    Returns:
        torch.Tensor: The similarity between the two molecules.
    """
    
    # Apply mask if provided
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]
    
    # Compute similarity by comparing the one-hot encoded features
    similarity = torch.sum(mol1[:,3:-1] == mol2[:,3:-1]) / mol1[:, 3:-1].numel()
    
    return similarity

## Explainability

### Utility function for visualization purposes

In [13]:
def draw_sphere_xai(ax, x, y, z, size, color, alpha):
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    xs = size * np.outer(np.cos(u), np.sin(v))
    ys = size * np.outer(np.sin(u), np.sin(v)) #* 0.8
    zs = size * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x + xs, y + ys, z + zs, rstride=2, cstride=2, color=color, alpha=alpha)

def plot_molecule_xai(ax, positions, atom_type, alpha, spheres_3d, hex_bg_color, is_geom, fragment_mask=None, phi_values=None):
    x = positions[:, 0]
    y = positions[:, 1]
    z = positions[:, 2]
    # Hydrogen, Carbon, Nitrogen, Oxygen, Flourine

    idx2atom = const.GEOM_IDX2ATOM if is_geom else const.IDX2ATOM

    colors_dic = np.array(const.COLORS)
    radius_dic = np.array(const.RADII)
    area_dic = 1500 * radius_dic ** 2

    areas = area_dic[atom_type]
    radii = radius_dic[atom_type]
    colors = colors_dic[atom_type]

    if fragment_mask is None:
        fragment_mask = torch.ones(len(x))

    for i in range(len(x)):
        for j in range(i + 1, len(x)):
            p1 = np.array([x[i], y[i], z[i]])
            p2 = np.array([x[j], y[j], z[j]])
            dist = np.sqrt(np.sum((p1 - p2) ** 2))
            atom1, atom2 = idx2atom[atom_type[i]], idx2atom[atom_type[j]]
            draw_edge_int = get_bond_order(atom1, atom2, dist)
            line_width = (3 - 2) * 2 * 2
            draw_edge = draw_edge_int > 0
            if draw_edge:
                if draw_edge_int == 4:
                    linewidth_factor = 1.5
                else:
                    linewidth_factor = 1
                linewidth_factor *= 0.5
                ax.plot(
                    [x[i], x[j]], [y[i], y[j]], [z[i], z[j]],
                    linewidth=line_width * linewidth_factor * 2,
                    c=hex_bg_color,
                    alpha=alpha
                )

    # from pdb import set_trace
    # set_trace()

    if spheres_3d:
        # idx = torch.where(fragment_mask[:len(x)] == 0)[0]
        # ax.scatter(
        #     x[idx],
        #     y[idx],
        #     z[idx],
        #     alpha=0.9 * alpha,
        #     edgecolors='#FCBA03',
        #     facecolors='none',
        #     linewidths=2,
        #     s=900
        # )
        for i, j, k, s, c, f, phi in zip(x, y, z, radii, colors, fragment_mask, phi_values):
            if f == 1:
                alpha = 1.0
                if phi > 0:
                    c = 'red'

            draw_sphere_xai(ax, i.item(), j.item(), k.item(), 0.5 * s, c, alpha)

    else:
        phi_values_array = np.array(list(phi_values.values()))

        # #draw fragments
        # fragment_mask_on_cpu = fragment_mask.cpu().numpy()
        # colors_fragment = colors[fragment_mask_on_cpu == 1]
        # x_fragment = x[fragment_mask_on_cpu == 1]
        # y_fragment = y[fragment_mask_on_cpu == 1]
        # z_fragment = z[fragment_mask_on_cpu == 1]
        # areas_fragment = areas[fragment_mask_on_cpu == 1]
        # ax.scatter(x_fragment, y_fragment, z_fragment, s=areas_fragment, alpha=0.9 * alpha, c=np.where(phi_values_array > 0, 'red', colors_fragment))

        # #draw non-fragment atoms
        # colors = colors[fragment_mask_on_cpu == 0]
        # x = x[fragment_mask_on_cpu == 0]
        # y = y[fragment_mask_on_cpu == 0]
        # z = z[fragment_mask_on_cpu == 0]
        # areas = areas[fragment_mask_on_cpu == 0]
        # ax.scatter(x, y, z, s=areas, alpha=0.9 * alpha, c=colors)

        #draw fragments
        fragment_mask_on_cpu = fragment_mask.cpu().numpy()
        colors_fragment = colors[fragment_mask_on_cpu == 1]
        x_fragment = x[fragment_mask_on_cpu == 1]
        y_fragment = y[fragment_mask_on_cpu == 1]
        z_fragment = z[fragment_mask_on_cpu == 1]
        areas_fragment = areas[fragment_mask_on_cpu == 1]
        
        # Calculate the gradient colors based on phi values
        cmap = plt.cm.get_cmap('coolwarm')
        norm = plt.Normalize(vmin=min(phi_values_array), vmax=max(phi_values_array))
        colors_fragment_shadow = cmap(norm(phi_values_array))
        
        # ax.scatter(x_fragment, y_fragment, z_fragment, s=areas_fragment, alpha=0.9 * alpha, c=colors_fragment)

        ax.scatter(x_fragment, y_fragment, z_fragment, s=areas_fragment, alpha=0.9 * alpha, c=colors_fragment, edgecolors=colors_fragment_shadow, linewidths=5, rasterized=False)

        #draw non-fragment atoms
        colors = colors[fragment_mask_on_cpu == 0]
        x = x[fragment_mask_on_cpu == 0]
        y = y[fragment_mask_on_cpu == 0]
        z = z[fragment_mask_on_cpu == 0]
        areas = areas[fragment_mask_on_cpu == 0]
        ax.scatter(x, y, z, s=areas, alpha=0.9 * alpha, c=colors, rasterized=False)


def plot_data3d_xai(positions, atom_type, is_geom, camera_elev=0, camera_azim=0, save_path=None, spheres_3d=False,
                bg='black', alpha=1., fragment_mask=None, phi_values=None):
    black = (0, 0, 0)
    white = (1, 1, 1)
    hex_bg_color = '#FFFFFF' if bg == 'black' else '#000000' #'#666666'

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection='3d')
    ax.set_aspect('auto')
    ax.view_init(elev=camera_elev, azim=camera_azim)
    if bg == 'black':
        ax.set_facecolor(black)
    else:
        ax.set_facecolor(white)
    ax.xaxis.pane.set_alpha(0)
    ax.yaxis.pane.set_alpha(0)
    ax.zaxis.pane.set_alpha(0)
    ax._axis3don = False

    if bg == 'black':
        ax.w_xaxis.line.set_color("black")
    else:
        ax.w_xaxis.line.set_color("white")

    plot_molecule_xai(
        ax, positions, atom_type, alpha, spheres_3d, hex_bg_color, is_geom=is_geom, fragment_mask=fragment_mask, phi_values=phi_values
    )

    max_value = positions.abs().max().item()
    axis_lim = min(40, max(max_value / 1.5 + 0.3, 3.2))
    ax.set_xlim(-axis_lim, axis_lim)
    ax.set_ylim(-axis_lim, axis_lim)
    ax.set_zlim(-axis_lim, axis_lim)
    dpi = 300 if spheres_3d else 300 #it was 120 and 50

    if save_path is not None:
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0, dpi=dpi)
        # plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0, dpi=dpi, transparent=True)

        if spheres_3d:
            img = imageio.imread(save_path)
            img_brighter = np.clip(img * 1.4, 0, 255).astype('uint8')
            imageio.imsave(save_path, img_brighter)
    else:
        plt.show()
    plt.close()

def visualize_chain_xai(
        path, spheres_3d=False, bg="black", alpha=1.0, wandb=None, mode="chain", is_geom=False, fragment_mask=None, phi_values=None
):
    files = load_xyz_files(path)
    save_paths = []

    # Fit PCA to the final molecule – to obtain the best orientation for visualization
    positions, one_hot, charges = load_molecule_xyz(files[-1], is_geom=is_geom)
    pca = PCA(n_components=3)
    pca.fit(positions)

    for i in range(len(files)):
        file = files[i]

        positions, one_hot, charges = load_molecule_xyz(file, is_geom=is_geom)
        atom_type = torch.argmax(one_hot, dim=1).numpy()

        # Transform positions of each frame according to the best orientation of the last frame
        positions = pca.transform(positions)
        positions = torch.tensor(positions)

        fn = file[:-4] + '.png'
        plot_data3d_xai(
            positions, atom_type,
            save_path=fn,
            spheres_3d=spheres_3d,
            alpha=alpha,
            bg=bg,
            camera_elev=90,
            camera_azim=90,
            is_geom=is_geom,
            fragment_mask=fragment_mask,
            phi_values=phi_values
        )
        save_paths.append(fn)

    imgs = [imageio.imread(fn) for fn in save_paths]
    dirname = os.path.dirname(save_paths[0])
    gif_path = dirname + '/output.gif'
    imageio.mimsave(gif_path, imgs, subrectangles=True)

    if wandb is not None:
        wandb.log({mode: [wandb.Video(gif_path, caption=gif_path)]})

### Explainabiliy phase

In [3]:

#@mastro
num_samples = 5
sampled = 0
#end @mastro
start = 0
bond_order_dict = {0:0, 1:0, 2:0, 3:0}
ATOM_SAMPLER = False
SAVE_VISUALIZATION = True
chain_with_full_fragments = None
M = 100 #number of Monte Carlo Sampling steps
P = 0.2 #probability of atom to exist in random graph (also edge in the future)
SEED = 42 #seed for random sampling
# Create the folder if it does not exist
folder_save_path = "results/explanations"
if not os.path.exists(folder_save_path):
    os.makedirs(folder_save_path)

for data in dataloader:
    
    if sampled < num_samples:
        chain_with_full_fragments = None
        sampled += 1
        
        # generate chain with original and full fragments

        chain_batch, node_mask = model.sample_chain(data, keep_frames=args.keep_frames)
        
        #get the generated molecule and store it in a variable
        chain_with_full_fragments = chain_batch[:, 0, :, :]

        # Compute distance of two chains
        mol_similarity = compute_molecular_similarity(chain_with_full_fragments.squeeze(), chain_with_full_fragments.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data["linker_mask"][0].squeeze())
        print("Similarity between the two chains:", mol_similarity.item())
        # compute similarity of one-hot vectors
        positional_similarity = compute_molecular_similarity_positions(chain_with_full_fragments.squeeze(), chain_with_full_fragments.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data["linker_mask"][0].squeeze())
        print("Similarity between the two chains based on positions:", positional_similarity.item())
        one_hot_similarity = compute_one_hot_similarity(chain_with_full_fragments.squeeze(), chain_with_full_fragments.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data["linker_mask"][0].squeeze())
        print("Similarity between the two one-hot vectors:", one_hot_similarity.item())
        # compute cosine similarity
        cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_with_full_fragments.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data["linker_mask"][0].squeeze().cpu())
        print("Cosine similarity between the two chains:", cosine_similarity)
        
    
        
        # display(data["fragment_mask"])
        # display(data["fragment_mask"].shape)

        # display(data["linker_mask"])
        # display(data["linker_mask"].shape)
        
        # display(data["edge_mask"])
        # display(data["edge_mask"].shape)

        #mask out all edges that are not bonds
        # idx2atom = const.GEOM_IDX2ATOM if model.is_geom else const.IDX2ATOM
      
        # positions = data["positions"][0].detach().cpu().numpy()
        # x  = positions[:,0]
        # y  = positions[:,1]
        # z  = positions[:,2]
        # # print(x)
       
        # atom_type = torch.argmax(data["one_hot"][0], dim=1)
        # print("Number of edges", len(x) * len(x))
        # sys.exit()
        #uncomment to work on edge_mask (not huge effect, tho)
        # for i in range(len(x)):
        #     for j in range(i+1, len(x)):
        #         p1 = np.array([x[i], y[i], z[i]])
        #         p2 = np.array([x[j], y[j], z[j]])
        #         dist =  np.sqrt(np.sum((p1 - p2) ** 2)) #np.linalg.norm(p1-p2)
                
        #         atom1, atom2 = idx2atom[atom_type[i].item()], idx2atom[atom_type[j].item()]
        #         bond_order = get_bond_order(atom1, atom2, dist)
                
        #         bond_order_dict[bond_order] += 1
        #         # if bond_order <= 0: #TODO debug. Why not all set to 0?
        #         if True:
        #             data["edge_mask"][i * len(x) + j] = 0
        #             data["edge_mask"][j * len(x) + i] = 0
        #         #set all edge_mask indices to 0
        #         data["edge_mask"] = torch.zeros_like(data["edge_mask"])

        #randomly mask out 50% of atoms
        # mask = torch.rand(data["atom_mask"].shape) > 0.5
        # data["atom_mask"] = data["atom_mask"] * mask.to(model.device)
        #mask out all atoms
        # data["atom_mask"] = torch.zeros_like(data["atom_mask"])
        
        #variables that will become function/class arguments/variables

        
        num_fragment_atoms = torch.sum(data["fragment_mask"] == 1)

        rng = default_rng(seed = SEED)
        phi_atoms = {}
        fragment_indices = torch.where(data["fragment_mask"] == 1)[1]
        num_fragment_atoms = len(fragment_indices)
        num_atoms = data["positions"].shape[1]

        distances_random_samples = []
        cosine_similarities_random_samples = []

        for j in tqdm(range(num_fragment_atoms)):
            
            marginal_contrib_distance = 0
            marginal_contrib_cosine_similarity = 0
            marginal_contrib_hausdorff = 0

            for step in tqdm(range(M)):
                data_j_plus = data.copy()
                data_j_minus = data.copy()
                data_random = data.copy()

                N_z_mask = rng.binomial(1, P, size = num_fragment_atoms)

                # Ensure at least one element is 1, otherwise randomly select one since at least one fragment atom must be present
                if not np.any(N_z_mask):
                    print("Zero elements in N_z_mask, randomly selecting one.")
                    random_index = rng.integers(0, num_fragment_atoms)
                    N_z_mask[random_index] = 1

                # print("N_z_mask for sample", sampled, step, N_z_mask)

                N_mask = torch.ones(num_fragment_atoms, dtype=torch.int)

                pi = torch.randperm(num_fragment_atoms)

                N_j_plus_index = torch.ones(num_fragment_atoms, dtype=torch.int)
                N_j_minus_index = torch.ones(num_fragment_atoms, dtype=torch.int)
                selected_node_index = np.where(pi == j)[0].item()
                
                # print("Selected node index", selected_node_index)
                for k in range(num_fragment_atoms):
                    if k <= selected_node_index:
                        N_j_plus_index[pi[k]] = N_mask[pi[k]]
                    else:
                        N_j_plus_index[pi[k]] = N_z_mask[pi[k]]

                for k in range(num_fragment_atoms):
                    if k < selected_node_index:
                        N_j_minus_index[pi[k]] = N_mask[pi[k]]
                    else:
                        N_j_minus_index[pi[k]] = N_z_mask[pi[k]]


                # print("N_j_plus_index", N_j_plus_index)
                # print("N_j_minus_index", N_j_minus_index)
                # print(N_j_plus_index == N_j_minus_index)
                
                N_j_plus = fragment_indices[N_j_plus_index.bool()] #fragement indices to keep in molecule j plus
                N_j_minus = fragment_indices[N_j_minus_index.bool()] #fragement indices to keep in molecule j minus

                N_random_sample = fragment_indices[torch.IntTensor(N_z_mask).bool()] #fragement indices to keep in random molecule
                # print("N_j_plus", N_j_plus)
                # print("N_j_minus", N_j_minus)
                # print(N_j_plus == N_j_minus)
                atom_mask_j_plus = torch.zeros(num_atoms, dtype=torch.bool)
                atom_mask_j_minus = torch.zeros(num_atoms, dtype=torch.bool)

                atom_mask_random_molecule = torch.zeros(num_atoms, dtype=torch.bool)

                atom_mask_j_plus[N_j_plus] = True
                #set to true also linker atoms
                atom_mask_j_plus[data["linker_mask"][0].squeeze().to(torch.int) == 1] = True
                atom_mask_j_minus[N_j_minus] = True
                #set to true also linker atoms
                atom_mask_j_minus[data["linker_mask"][0].squeeze().to(torch.int) == 1] = True

                atom_mask_random_molecule[N_random_sample] = True
                #set to true also linker atoms
                atom_mask_random_molecule[data["linker_mask"][0].squeeze().to(torch.int) == 1] = True

                # print("Atom mask j plus", atom_mask_j_plus)
                # print("Atom mask j minus", atom_mask_j_minus)
                # print(atom_mask_j_minus==atom_mask_j_plus)

                #for sample containing j
                #remove positions of atoms in random_indices
                data_j_plus["positions"] = data_j_plus["positions"][:, atom_mask_j_plus]
                #remove one_hot of atoms in random_indices
                data_j_plus["one_hot"] = data_j_plus["one_hot"][:, atom_mask_j_plus]
                #remove atom_mask of atoms in random_indices
                data_j_plus["atom_mask"] = data_j_plus["atom_mask"][:, atom_mask_j_plus]
                #remove fragment_mask of atoms in random_indices
                data_j_plus["fragment_mask"] =  data_j_plus["fragment_mask"][:, atom_mask_j_plus]
                #remove linker_mask of atoms in random_indices
                data_j_plus["linker_mask"] = data_j_plus["linker_mask"][:, atom_mask_j_plus]
                #remove edge_mask of atoms in random_indices
                for index in N_j_plus:
                    for i in range(num_atoms):
                        data_j_plus["edge_mask"][index * num_atoms + i] = 0
                        data_j_plus["edge_mask"][i * num_atoms + index] = 0

                #remove all values in edge_mask that are 0
                data_j_plus["edge_mask"] = data_j_plus["edge_mask"][data_j_plus["edge_mask"] != 0]  #to be checked, but working on atoms has as effect. For the moment we stick to atoms, then we move to edges (need to edit internal function for this, or redefine everything...)

                # print("After removal j plus:", data_j_plus["positions"])
                # print(data_j_plus["positions"].shape)
                
                #for sample not containing j
                #remove positions of atoms in random_indices
                data_j_minus["positions"] = data_j_minus["positions"][:, atom_mask_j_minus]
                #remove one_hot of atoms in random_indices
                data_j_minus["one_hot"] = data_j_minus["one_hot"][:, atom_mask_j_minus]
                #remove atom_mask of atoms in random_indices
                data_j_minus["atom_mask"] = data_j_minus["atom_mask"][:, atom_mask_j_minus]
                #remove fragment_mask of atoms in random_indices
                data_j_minus["fragment_mask"] =  data_j_minus["fragment_mask"][:, atom_mask_j_minus]
                #remove linker_mask of atoms in random_indices
                data_j_minus["linker_mask"] = data_j_minus["linker_mask"][:, atom_mask_j_minus]
                #remove edge_mask of atoms in random_indices
                for index in N_j_minus:
                    for i in range(num_atoms):
                        data_j_minus["edge_mask"][index * num_atoms + i] = 0
                        data_j_minus["edge_mask"][i * num_atoms + index] = 0

                #remove all values in edge_mask that are 0
                data_j_minus["edge_mask"] = data_j_minus["edge_mask"][data_j_minus["edge_mask"] != 0]  #to be checked, but working on atoms has as effect. For the moment we stick to atoms, then we move to edges (need to edit internal function for this, or redefine everything...)

                # print("After removal j minus:", data_j_minus["positions"])
                # print(data_j_minus["positions"].shape)

                #for random sample
                data_random["positions"] = data_random["positions"][:, atom_mask_random_molecule]
                #remove one_hot of atoms in random_indices
                data_random["one_hot"] = data_random["one_hot"][:, atom_mask_random_molecule]
                #remove atom_mask of atoms in random_indices
                data_random["atom_mask"] = data_random["atom_mask"][:, atom_mask_random_molecule]
                #remove fragment_mask of atoms in random_indices
                data_random["fragment_mask"] =  data_random["fragment_mask"][:, atom_mask_random_molecule]
                #remove linker_mask of atoms in random_indices
                data_random["linker_mask"] = data_random["linker_mask"][:, atom_mask_random_molecule]
                #remove edge_mask of atoms in random_indices
                for index in N_z_mask:
                    for i in range(num_atoms):
                        data_random["edge_mask"][index * num_atoms + i] = 0
                        data_random["edge_mask"][i * num_atoms + index] = 0

                #remove all values in edge_mask that are 0
                data_random["edge_mask"] = data_random["edge_mask"][data_random["edge_mask"] != 0] 



                #with node j
                chain_j_plus, node_mask_j_plus = model.sample_chain(data_j_plus, keep_frames=args.keep_frames)

                V_j_plus_distance = compute_molecular_distance(chain_with_full_fragments.squeeze(), chain_j_plus.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data_j_plus["linker_mask"][0].squeeze())

                V_j_plus_cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_j_plus.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data_j_plus["linker_mask"][0].squeeze().cpu())

                # print("V_j_plus", V_j_plus)

                #without node j
                chain_j_minus, node_mask_j_minus = model.sample_chain(data_j_minus, keep_frames=args.keep_frames)

                V_j_minus_distance = compute_molecular_distance(chain_with_full_fragments.squeeze(), chain_j_minus.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data_j_minus["linker_mask"][0].squeeze())

                V_j_minus_cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_j_minus.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data_j_minus["linker_mask"][0].squeeze().cpu())

                #with random sample
                chain_random, node_mask_random = model.sample_chain(data_random, keep_frames=args.keep_frames)

                V_random_distance = compute_molecular_distance(chain_with_full_fragments.squeeze(), chain_random.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data_random["linker_mask"][0].squeeze())

                V_random_cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_random.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data_random["linker_mask"][0].squeeze().cpu())

                distances_random_samples.append(V_random_distance)
                cosine_similarities_random_samples.append(V_random_cosine_similarity)

                # print(V_random_distance, V_random_cosine_similarity)
                
                marginal_contrib_distance += (V_j_plus_distance - V_j_minus_distance)

                marginal_contrib_cosine_similarity += (V_j_plus_cosine_similarity - V_j_minus_cosine_similarity)

                # marginal_contrib_hausdorff += (V_j_plus_hausdorff - V_j_minus_hausdorff)

            phi_atoms[fragment_indices[j].item()] = [0,0] #,0]    
            phi_atoms[fragment_indices[j].item()][0] = marginal_contrib_distance/M #j is the index of the fragment atom in the fragment indices tensor
            phi_atoms[fragment_indices[j].item()][1] = marginal_contrib_cosine_similarity/M
            # phi_atoms[fragment_indices[j]][2] = marginal_contrib_hausdorff/M

            print(data["name"])

        phi_atoms_distances = {}
        phi_atoms_cosine_similarity = {}
        for atom_index, phi_values in phi_atoms.items():
            phi_atoms_distances[atom_index] = phi_values[0]
            phi_atoms_cosine_similarity[atom_index] = phi_values[1]

        if SAVE_VISUALIZATION:
            for i in range(len(data['positions'])):
                chain = chain_batch[:, i, :, :]
                assert chain.shape[0] == args.keep_frames
                assert chain.shape[1] == data['positions'].shape[1]
                assert chain.shape[2] == data['positions'].shape[2] + data['one_hot'].shape[2] + model.include_charges

                # Saving chains
                name = str(i + start)
                chain_output = os.path.join(chains_output_dir, name)
                os.makedirs(chain_output, exist_ok=True)

                one_hot = chain[:, :, 3:-1]
                positions = chain[:, :, :3]
                chain_node_mask = torch.cat([node_mask[i].unsqueeze(0) for _ in range(args.keep_frames)], dim=0)
                names = [f'{name}_{j}' for j in range(args.keep_frames)]

                save_xyz_file(chain_output, one_hot, positions, chain_node_mask, names=names, is_geom=model.is_geom)
                visualize_chain_xai(
                    chain_output,
                    spheres_3d=True,
                    alpha=0.7,
                    bg='white',
                    is_geom=model.is_geom,
                    fragment_mask=data['fragment_mask'][i].squeeze(),
                    phi_values=phi_atoms_distances
                )

                # Saving final prediction and ground truth separately
                true_one_hot = data['one_hot'][i].unsqueeze(0)
                true_positions = data['positions'][i].unsqueeze(0)
                true_node_mask = data['atom_mask'][i].unsqueeze(0)
                save_xyz_file(
                    final_states_output_dir,
                    true_one_hot,
                    true_positions,
                    true_node_mask,
                    names=[f'{name}_true'],
                    is_geom=model.is_geom,
                )

                pred_one_hot = chain[0, :, 3:-1].unsqueeze(0)
                pred_positions = chain[0, :, :3].unsqueeze(0)
                pred_node_mask = chain_node_mask[0].unsqueeze(0)
                save_xyz_file(
                    final_states_output_dir,
                    pred_one_hot,
                    pred_positions,
                    pred_node_mask,
                    names=[f'{name}_pred'],
                    is_geom=model.is_geom
                )

            start += len(data['positions'])

        # Save phi_atoms to a text file
        with open(f'{folder_save_path}/phi_atoms_{sampled}.txt', 'w') as write_file:
            write_file.write("sample name: " + str(data["name"]) + "\n")
            write_file.write("atom_index,distance,cosine_similarity\n")
            for atom_index, phi_values in phi_atoms.items():
                write_file.write(f"{atom_index},{phi_values[0]},{phi_values[1]}\n")

            write_file.write("\n")      
            write_file.write("Distance random samples\n")
            write_file.write(str(distances_random_samples) + "\n")
            write_file.write("Cosine similarity random samples\n")
            write_file.write(str(cosine_similarities_random_samples) + "\n")
      


Similarity between the two chains: 1.0
Similarity between the two chains based on positions: 1.0
Similarity between the two one-hot vectors: 1.0


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# if ATOM_SAMPLER:
#             print("Before removal:", data["positions"].shape)
#             #get all indices in atom_mask that correspond to fragment atoms
#             fragment_indices = torch.where(data["fragment_mask"] == 1)[1]

#             print(fragment_indices)
            
#             #randomly pick 50% of fragment atoms
#             random_indices = torch.randperm(len(fragment_indices))[:int(len(fragment_indices)/2)]
#             mask_fragments = torch.ones(len(fragment_indices), dtype=torch.bool)
#             mask_fragments[random_indices] = False
#             selected_fragment_atoms = fragment_indices[mask_fragments]

#             #keep only the first index in selected_fragment_atoms
#             selected_fragment_atoms = selected_fragment_atoms[:1] #@mastro change this removing the explained atom
#             print("Selected fragment atoms", selected_fragment_atoms)
            
#             num_atoms = data["positions"].shape[1]
#             # random_indices = torch.randperm(num_atoms)[:int(num_atoms/2)]
#             mask = torch.ones(num_atoms, dtype=torch.bool)
#             mask[selected_fragment_atoms] = False


#             #remove positions of atoms in random_indices
#             data["positions"] = data["positions"][:, mask]
#             #remove one_hot of atoms in random_indices
#             data["one_hot"] = data["one_hot"][:, mask]
#             #remove atom_mask of atoms in random_indices
#             data["atom_mask"] = data["atom_mask"][:, mask]
#             #remove fragment_mask of atoms in random_indices
#             data["fragment_mask"] =  data["fragment_mask"][:, mask]
#             #remove linker_mask of atoms in random_indices
#             data["linker_mask"] = data["linker_mask"][:, mask]
#             #remove edge_mask of atoms in random_indices
#             for index in random_indices:
#                 for i in range(num_atoms):
#                     data["edge_mask"][index * num_atoms + i] = 0
#                     data["edge_mask"][i * num_atoms + index] = 0

#             #remove all values in edge_mask that are 0
#             data["edge_mask"] = data["edge_mask"][data["edge_mask"] != 0]  #to be checked, but working on atoms has as effect. For the moment we stick to atoms, then we move to edges (need to edit internal function for this, or redefine everything...)
            

#             print("After removal:", data["positions"].shape)
#             # sys.exit()
#             # print number of zeros in edge mask
#             print("Number of masked out edges (edges not representing bonds)", torch.sum(data["edge_mask"] == 0))
#             print("Number of edges still present", torch.sum(data["edge_mask"] != 0))

#             # print number of zeros in atom mask
#             print("Number of masked out atoms", torch.sum(data["atom_mask"] == 0))

## Visualization - trials

Implementing visualization highlighting important atoms

In [12]:
# phi_atoms = {0: -0.1030,
#  1: -2.5306,
#  2: 0.3042,
#  3: -1.4719,
#  4: -1.3421,
#  5: -0.2840,
#  6: 0.9049,
#  7: -1.1589,
#  8: -0.2158,
#  9: -1.8067,
#  10: -2.1304,
#  11: -0.2276,
#  12: -1.6576,
#  13: -0.0887,
#  14: 0.0418,
#  15: -1.8112,
#  16: 0.4877,
#  17: 0.0786,
#  18: -0.2415,
#  19: -1.5389,
#  20: -1.8475}

# shapley_values = {0:(-0.890280568599701,0.06123639730736613),
# 1:(0.18284012019634246,-0.0529984375834465),
# 2:(-4.252275815010071,0.061238834373652934),
# 3:(-2.081632282733917,0.26110127678141),
# 4:(0.01699479103088379,-0.013148795943707228),
# 5:(-0.24424300074577332,0.044520441107451916),
# 6:(0.5019020104408264,-0.06629922293126583),
# 7:(0.39756537079811094,-0.04101886671036482),
# 8:(0.2415522527694702,0.0025899429991841317),
# 9:(-1.1490421509742736,0.17777293106541037),
# 10:(-1.3627109980583192,0.22668330844491721),
# 11:(-0.17836040258407593,0.03489074512384832),
# 12:(-1.182363440990448,0.11668485717847943),
# 13:(0.34586033344268796,-0.02710286658257246),
# 14:(-3.1875360774993897,0.04069803573191166),
# 15:(-0.07272281646728515,0.019701205156743527),
# 16:(-0.9446532082557678,-0.05654389940202236),
# 17:(-4.32677052974701,0.10536605073139071),
# 18:(-0.32534114360809324,-0.0567812330275774),
# 19:(-0.345893794298172,0.029798283874988556),
# 20:(0.19235227584838868,-0.014152660053223371)}

# phi_atoms_distances = {}
# phi_atoms_cosine_similarity = {}

# for atom_index, phi_values in shapley_values.items():
#     phi_atoms_distances[atom_index] = phi_values[0]
#     phi_atoms_cosine_similarity[atom_index] = phi_values[1]

# print(phi_atoms_distances)
# print(phi_atoms_cosine_similarity)

{0: -0.890280568599701, 1: 0.18284012019634246, 2: -4.252275815010071, 3: -2.081632282733917, 4: 0.01699479103088379, 5: -0.24424300074577332, 6: 0.5019020104408264, 7: 0.39756537079811094, 8: 0.2415522527694702, 9: -1.1490421509742736, 10: -1.3627109980583192, 11: -0.17836040258407593, 12: -1.182363440990448, 13: 0.34586033344268796, 14: -3.1875360774993897, 15: -0.07272281646728515, 16: -0.9446532082557678, 17: -4.32677052974701, 18: -0.32534114360809324, 19: -0.345893794298172, 20: 0.19235227584838868}
{0: 0.06123639730736613, 1: -0.0529984375834465, 2: 0.061238834373652934, 3: 0.26110127678141, 4: -0.013148795943707228, 5: 0.044520441107451916, 6: -0.06629922293126583, 7: -0.04101886671036482, 8: 0.0025899429991841317, 9: 0.17777293106541037, 10: 0.22668330844491721, 11: 0.03489074512384832, 12: 0.11668485717847943, 13: -0.02710286658257246, 14: 0.04069803573191166, 15: 0.019701205156743527, 16: -0.05654389940202236, 17: 0.10536605073139071, 18: -0.0567812330275774, 19: 0.029798283

#### Utility functions for visualization modified from DiffLinker work

In [5]:
# phi_values_array = np.array(list(phi_atoms.values()))
# phi_values_array.shape

(21,)

### Visualization step

In [15]:
# start = 0
# for data in tqdm(dataloader):
#     chain_batch, node_mask = model.sample_chain(data, keep_frames=args.keep_frames)
#     for i in tqdm(range(len(data['positions']))):
#         chain = chain_batch[:, i, :, :]
#         assert chain.shape[0] == args.keep_frames
#         assert chain.shape[1] == data['positions'].shape[1]
#         assert chain.shape[2] == data['positions'].shape[2] + data['one_hot'].shape[2] + model.include_charges

#         # Saving chains
#         name = str(i + start)
#         chain_output = os.path.join(chains_output_dir, name)
#         os.makedirs(chain_output, exist_ok=True)

#         one_hot = chain[:, :, 3:-1]
#         positions = chain[:, :, :3]
#         chain_node_mask = torch.cat([node_mask[i].unsqueeze(0) for _ in range(args.keep_frames)], dim=0)
#         names = [f'{name}_{j}' for j in range(args.keep_frames)]

#         save_xyz_file(chain_output, one_hot, positions, chain_node_mask, names=names, is_geom=model.is_geom)
#         visualize_chain_xai(
#             chain_output,
#             spheres_3d=False,
#             alpha=0.7,
#             bg='white',
#             is_geom=model.is_geom,
#             fragment_mask=data['fragment_mask'][i].squeeze(),
#             phi_values=phi_atoms_cosine_similarity
#         )

#         # Saving final prediction and ground truth separately
#         true_one_hot = data['one_hot'][i].unsqueeze(0)
#         true_positions = data['positions'][i].unsqueeze(0)
#         true_node_mask = data['atom_mask'][i].unsqueeze(0)
#         save_xyz_file(
#             final_states_output_dir,
#             true_one_hot,
#             true_positions,
#             true_node_mask,
#             names=[f'{name}_true'],
#             is_geom=model.is_geom,
#         )

#         pred_one_hot = chain[0, :, 3:-1].unsqueeze(0)
#         pred_positions = chain[0, :, :3].unsqueeze(0)
#         pred_node_mask = chain_node_mask[0].unsqueeze(0)
#         save_xyz_file(
#             final_states_output_dir,
#             pred_one_hot,
#             pred_positions,
#             pred_node_mask,
#             names=[f'{name}_pred'],
#             is_geom=model.is_geom
#         )

#     start += len(data['positions'])
#     break

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Mastro\AppData\Local\Temp\ipykernel_21748\1457623096.py:203: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(fn) for fn in save_paths]


In [ ]:
# import numpy as np

# chain_with_full_fragments = chain_with_full_fragments[0,:,:]
# # Generate a random boolean mask
# mask = np.random.choice([True, False], size=chain_with_full_fragments.shape[0])

# # Apply the mask to remove random rows
# masked_chain = chain_with_full_fragments[mask, :]

# masked_chain.shape

In [ ]:
# for data in dataloader:
#     positions = data["positions"]
#     # print(positions)
    
#     cos_sim = cosine_similarity((positions[0].squeeze().detach().cpu().numpy().flatten().reshape(1, -1)),positions[0].squeeze().detach().cpu().numpy().flatten().reshape(1, -1))

#     cosine = compute_cosine_similarity(positions[0].squeeze().cpu(), positions[0].squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data["linker_mask"][0].squeeze().cpu())
#     print(cosine)
#     break

In [ ]:
# atom_mask_random_molecule

## Checking Shapley value Propeties

In [15]:
shapley_values = {0:(-0.890280568599701,0.06123639730736613),
1:(0.18284012019634246,-0.0529984375834465),
2:(-4.252275815010071,0.061238834373652934),
3:(-2.081632282733917,0.26110127678141),
4:(0.01699479103088379,-0.013148795943707228),
5:(-0.24424300074577332,0.044520441107451916),
6:(0.5019020104408264,-0.06629922293126583),
7:(0.39756537079811094,-0.04101886671036482),
8:(0.2415522527694702,0.0025899429991841317),
9:(-1.1490421509742736,0.17777293106541037),
10:(-1.3627109980583192,0.22668330844491721),
11:(-0.17836040258407593,0.03489074512384832),
12:(-1.182363440990448,0.11668485717847943),
13:(0.34586033344268796,-0.02710286658257246),
14:(-3.1875360774993897,0.04069803573191166),
15:(-0.07272281646728515,0.019701205156743527),
16:(-0.9446532082557678,-0.05654389940202236),
17:(-4.32677052974701,0.10536605073139071),
18:(-0.32534114360809324,-0.0567812330275774),
19:(-0.345893794298172,0.029798283874988556),
20:(0.19235227584838868,-0.014152660053223371)}

In [17]:
sum_shapley_values_distance = 0
sum_shapley_values_cosine_similarity = 0

for key, value in shapley_values.items():
    sum_shapley_values_distance += value[0]
    sum_shapley_values_cosine_similarity += value[1]

print("Sum of shapley values for distance", sum_shapley_values_distance)
print("Sum of shapley values for cosine similarity", sum_shapley_values_cosine_similarity)

Sum of shapley values for distance -18.664759075045584
Sum of shapley values for cosine similarity 0.854236327642575


In [24]:
disances_random_graphs = [8.209129333496094, 8.952604293823242, 10.214823722839355, 13.954014778137207, 9.902146339416504, 7.748309135437012, 9.58663272857666, 8.814839363098145, 8.533931732177734, 10.899216651916504, 6.446581840515137, 11.022216796875, 10.328801155090332, 10.357083320617676, 13.72740364074707, 8.095475196838379, 9.76915454864502, 8.047905921936035, 15.773385047912598, 13.062732696533203, 13.816431999206543, 10.048111915588379, 8.194252967834473, 18.1801700592041, 10.82494068145752, 9.73255729675293, 57.117897033691406, 6.423393249511719, 7.342648983001709, 8.610400199890137, 7.232339859008789, 8.377412796020508, 9.754656791687012, 13.686880111694336, 7.862268447875977, 17.248708724975586, 10.300691604614258, 9.93982982635498, 11.898503303527832, 10.020878791809082, 17.05807876586914, 10.005925178527832, 11.349445343017578, 12.494544982910156, 9.537857055664062, 6.723769664764404, 9.319975852966309, 7.918763637542725, 9.330039024353027, 12.573590278625488, 8.421866416931152, 5.286699295043945, 10.08842945098877, 10.49685001373291, 8.781761169433594, 11.370782852172852, 9.893583297729492, 10.117992401123047, 7.283700466156006, 11.34451961517334, 10.696557998657227, 10.079591751098633, 6.065895080566406, 11.55136775970459, 5.719263076782227, 13.822511672973633, 9.270491600036621, 9.231322288513184, 9.714028358459473, 11.89116096496582, 7.7761735916137695, 10.131880760192871, 9.913873672485352, 28.074527740478516, 8.967206001281738, 8.669889450073242, 7.542867183685303, 13.192893028259277, 10.989325523376465, 9.27221965789795, 11.111801147460938, 12.508758544921875, 13.832940101623535, 9.58504867553711, 10.186914443969727, 9.9342679977417, 12.141968727111816, 9.668045043945312, 7.9129414558410645, 7.751343250274658, 5.762153148651123, 17.41439437866211, 12.621919631958008, 12.221673965454102, 9.3302583694458, 11.133764266967773, 11.901529312133789, 11.752213478088379, 12.65218734741211, 41.186737060546875, 8.035484313964844, 11.247607231140137, 9.454121589660645, 7.320855617523193, 10.293953895568848, 12.780405044555664, 39.704097747802734, 12.138388633728027, 9.946394920349121, 11.477827072143555, 9.735174179077148, 14.452847480773926, 11.18716812133789, 9.775537490844727, 9.246755599975586, 12.440742492675781, 32.06311798095703, 9.166975975036621, 12.411175727844238, 6.741437911987305, 10.38508129119873, 64.08743286132812, 10.278059005737305, 14.089242935180664, 9.274314880371094, 7.0374836921691895, 7.991494655609131, 11.855350494384766, 14.227992057800293, 9.37634563446045, 161.18458557128906, 7.345897674560547, 10.285453796386719, 6.564380168914795, 7.772758960723877, 13.130574226379395, 123.10112762451172, 9.865630149841309, 10.63464069366455, 62.38740539550781, 6.51915979385376, 7.852563858032227, 7.8991804122924805, 38.05455017089844, 14.546113014221191, 12.78609848022461, 13.193644523620605, 12.97070598602295, 128.3959197998047, 12.340230941772461, 5.315558433532715, 11.739768981933594, 12.66809368133545, 9.16489315032959, 12.024100303649902, 13.835417747497559, 56.344757080078125, 7.282908916473389, 11.958391189575195, 133.73951721191406, 12.320009231567383, 9.019210815429688, 8.31986141204834, 9.689737319946289, 10.767715454101562, 7.646640300750732, 11.75806713104248, 9.78736686706543, 10.653173446655273, 9.422762870788574, 12.769983291625977, 10.87141227722168, 10.187792778015137, 8.27980899810791, 14.046626091003418, 16.638710021972656, 7.49033260345459, 11.832338333129883, 12.526426315307617, 11.944979667663574, 8.587729454040527, 28.00351333618164, 126.24627685546875, 27.436506271362305, 7.994054317474365, 9.747160911560059, 12.11359977722168, 11.636900901794434, 8.313720703125, 7.126744747161865, 34.26729202270508, 9.955571174621582, 12.423892974853516, 10.432228088378906, 17.236295700073242, 11.37733268737793, 10.780647277832031, 12.866958618164062, 9.576515197753906, 7.71250057220459, 17.3825626373291, 7.952789306640625, 12.466327667236328, 6.006149768829346, 13.414301872253418, 10.215463638305664, 10.5762300491333, 7.216629505157471, 13.818830490112305, 10.106532096862793, 153.48207092285156, 9.427762031555176, 10.940261840820312, 8.63996696472168, 47.36735534667969, 151.89439392089844, 8.146526336669922, 6.839532375335693, 7.926476955413818, 12.1903076171875, 10.785548210144043, 8.719393730163574, 7.349748134613037, 12.377729415893555, 86.52647399902344, 14.939933776855469, 11.656821250915527, 11.249430656433105, 10.740544319152832, 41.62876510620117, 61.71883773803711, 9.999479293823242, 32.48844528198242, 6.29309606552124, 8.995867729187012, 5.773918151855469, 35.70499038696289, 10.788060188293457, 143.80975341796875, 12.347596168518066, 10.493147850036621, 10.572213172912598, 10.876154899597168, 9.234980583190918, 10.143120765686035, 10.068280220031738, 10.17026424407959, 11.171039581298828, 66.309814453125, 10.97315788269043, 8.941093444824219, 7.328882694244385, 7.639470100402832, 14.436507225036621, 6.37609338760376, 16.29436683654785, 6.866601467132568, 4.958314895629883, 4.613890171051025, 11.189050674438477, 9.602662086486816, 10.02652359008789, 18.929391860961914, 10.501829147338867, 8.310654640197754, 12.403564453125, 11.180450439453125, 128.0644073486328, 65.12410736083984, 6.514279365539551, 11.419157028198242, 6.953762531280518, 7.174751281738281, 6.047908782958984, 7.645975112915039, 8.493325233459473, 76.6259536743164, 51.15602493286133, 6.409876346588135, 139.14231872558594, 13.233312606811523, 12.61125659942627, 7.732011795043945, 141.91305541992188, 6.673437595367432, 12.75169563293457, 11.93539047241211, 12.067673683166504, 6.298254489898682, 12.425535202026367, 11.013300895690918, 9.617471694946289, 11.136137962341309, 13.480611801147461, 6.964269638061523, 15.643038749694824, 7.972656726837158, 10.877312660217285, 7.64153528213501, 13.512681007385254, 10.4829740524292, 8.181068420410156, 9.791838645935059, 10.944271087646484, 7.544434547424316, 6.5537028312683105, 14.936529159545898, 14.349370002746582, 11.657234191894531, 9.635701179504395, 9.449853897094727, 5.6097869873046875, 12.188665390014648, 8.897873878479004, 78.26348114013672, 5.783307075500488, 12.648677825927734, 14.524711608886719, 8.139593124389648, 9.616781234741211, 7.302402019500732, 9.041569709777832, 10.58753776550293, 9.137394905090332, 7.833139419555664, 8.797571182250977, 10.30905818939209, 11.062904357910156, 8.184062957763672, 9.72717571258545, 7.097469329833984, 5.7879252433776855, 5.287870407104492, 8.16462230682373, 10.81126594543457, 6.095418930053711, 5.451373100280762, 8.779481887817383, 9.44882869720459, 7.656450271606445, 10.422284126281738, 10.67720890045166, 12.994650840759277, 100.23351287841797, 8.253536224365234, 9.874190330505371, 7.635767459869385, 14.445303916931152, 11.892738342285156, 6.00723934173584, 9.232211112976074, 9.710413932800293, 12.725495338439941, 9.479235649108887, 9.667986869812012, 11.415745735168457, 7.233422756195068, 33.45899963378906, 11.954602241516113, 7.1619791984558105, 10.14799976348877, 12.242918968200684, 10.957411766052246, 13.038932800292969, 14.428964614868164, 6.619930267333984, 197.9977264404297, 13.079307556152344, 6.068120956420898, 11.337723731994629, 12.720364570617676, 8.236785888671875, 7.656992435455322, 33.229923248291016, 158.95750427246094, 10.327567100524902, 12.052824974060059, 30.92673683166504, 13.116118431091309, 7.110420227050781, 12.042588233947754, 12.301836013793945, 11.555768966674805, 5.867687702178955, 8.145340919494629, 8.333128929138184, 15.376466751098633, 10.6836576461792, 7.7100605964660645, 11.7132568359375, 7.1720356941223145, 8.95140552520752, 9.806631088256836, 14.461511611938477, 13.216554641723633, 10.785348892211914, 8.039161682128906, 12.58931827545166, 9.648996353149414, 28.72307014465332, 9.811858177185059, 9.134822845458984, 10.105224609375, 13.165461540222168, 11.144474983215332, 10.22687816619873, 13.635647773742676, 9.5993013381958, 10.750824928283691, 14.800241470336914, 14.085223197937012, 9.278888702392578, 5.463984966278076, 33.69841003417969, 86.2769775390625, 8.48144245147705, 11.273815155029297, 15.784255981445312, 6.80794620513916, 15.923543930053711, 10.192875862121582, 8.380311965942383, 10.489358901977539, 10.707287788391113, 14.279862403869629, 12.753467559814453, 64.08421325683594, 6.488614082336426, 4.951821327209473, 6.647948741912842, 6.941778182983398, 11.808536529541016, 47.38011169433594, 10.460448265075684, 9.843751907348633, 13.150506973266602, 10.801780700683594, 55.86006546020508, 9.284004211425781, 9.933599472045898, 15.063210487365723, 5.367635250091553, 9.553678512573242, 8.416173934936523, 12.76797103881836, 19.48387336730957, 7.405125617980957, 8.630844116210938, 10.761645317077637, 12.48518180847168, 11.641105651855469, 9.479816436767578, 7.663374423980713, 11.441161155700684, 9.772637367248535, 92.54299926757812, 10.383713722229004, 11.30550765991211, 9.066316604614258, 5.878917694091797, 13.175958633422852, 13.071769714355469, 13.19786548614502, 6.9518141746521, 11.696354866027832, 113.78341674804688, 58.56782531738281, 7.111131191253662, 9.273914337158203, 7.6878581047058105, 14.005342483520508, 10.286710739135742, 8.010111808776855, 9.106158256530762, 10.725813865661621, 7.321081638336182, 11.170199394226074, 9.2587890625, 10.452812194824219, 7.757209300994873, 11.548056602478027, 12.044645309448242, 14.720937728881836, 6.934476852416992, 11.320311546325684, 7.017487049102783, 158.6315460205078, 8.817571640014648, 9.826008796691895, 14.187663078308105, 9.468180656433105, 11.75041675567627, 8.779383659362793, 8.959869384765625, 11.022656440734863, 9.47191047668457, 7.284130573272705, 8.306800842285156, 7.406202793121338, 10.768976211547852, 13.128466606140137, 10.356640815734863, 7.751218318939209, 8.002760887145996, 5.2457146644592285, 10.299306869506836, 9.430336952209473, 14.712041854858398, 6.751877307891846, 9.959637641906738, 7.30118989944458, 14.2997465133667, 12.470906257629395, 10.463384628295898, 9.048273086547852, 138.9481201171875, 7.636450290679932, 10.3626708984375, 10.271665573120117, 11.782147407531738, 10.299490928649902, 14.968574523925781, 11.23881721496582, 9.84293270111084, 7.4602861404418945, 6.779901027679443, 8.994159698486328, 12.740147590637207, 7.077923774719238, 12.725715637207031, 11.464273452758789, 8.702104568481445, 9.486739158630371, 11.372965812683105, 11.398357391357422, 89.98810577392578, 11.347122192382812, 8.349882125854492, 9.860206604003906, 5.749568939208984, 10.955363273620605, 8.897257804870605, 12.372413635253906, 10.218037605285645, 12.74898910522461, 6.80877161026001, 6.293310165405273, 11.311748504638672, 9.633370399475098, 8.336322784423828, 6.709685325622559, 9.85675048828125, 8.216390609741211, 15.54067325592041, 12.265487670898438, 7.658849239349365, 7.149568557739258, 14.10133171081543, 14.42374324798584, 5.890360355377197, 14.898884773254395, 94.81440734863281, 14.542859077453613, 9.06374454498291, 8.756831169128418, 11.407970428466797, 29.391138076782227, 14.475071907043457, 10.280271530151367, 7.186238765716553, 11.626527786254883, 10.659919738769531, 11.737669944763184, 7.081657409667969, 9.742506980895996, 7.523128986358643, 77.50833129882812, 8.236814498901367, 9.616229057312012, 13.179754257202148, 65.87628936767578, 16.952756881713867, 8.589909553527832, 156.88192749023438, 12.767097473144531, 11.586220741271973, 14.710824012756348, 11.84715461730957, 7.214468479156494, 206.24998474121094, 5.639718055725098, 11.17142391204834, 5.360092639923096, 67.58735656738281, 11.99510383605957, 9.314101219177246, 8.335756301879883, 13.37955379486084, 8.048100471496582, 12.639413833618164, 9.012475967407227, 11.42516803741455, 92.89202880859375, 13.681418418884277, 8.569865226745605, 27.38974952697754, 7.2692179679870605, 7.777629852294922, 157.6053924560547, 13.123281478881836, 8.095538139343262, 7.596492767333984, 14.461600303649902, 13.980283737182617, 10.844585418701172, 9.892133712768555, 10.327054023742676, 13.334672927856445, 8.121040344238281, 9.44857120513916, 11.63930606842041, 5.916975021362305, 12.443222999572754, 13.610576629638672, 7.8992509841918945, 8.708757400512695, 11.954051971435547, 15.110066413879395, 12.205883979797363, 15.963156700134277, 10.560359001159668, 8.68052864074707, 10.335551261901855, 7.454211711883545, 13.31031322479248, 11.507926940917969, 9.301657676696777, 12.629115104675293, 10.674946784973145, 12.592028617858887, 9.68017578125, 16.339208602905273, 8.082587242126465, 9.843744277954102, 14.786746978759766, 11.611977577209473, 4.773707866668701, 8.786166191101074, 10.436694145202637, 101.97405242919922, 8.376087188720703, 165.4856414794922, 12.013035774230957, 13.814455032348633, 8.031562805175781, 10.615781784057617, 9.93582820892334, 11.567780494689941, 7.073474407196045, 5.5068583488464355, 11.572501182556152, 11.044428825378418, 8.301960945129395, 9.539912223815918, 8.466510772705078, 7.309288501739502, 14.173935890197754, 49.99473190307617, 12.942486763000488, 18.049423217773438, 141.69158935546875, 7.075711727142334, 17.055084228515625, 8.365362167358398, 9.829293251037598, 7.229154586791992, 10.878255844116211, 12.106363296508789, 9.087725639343262, 10.669891357421875, 15.68321418762207, 6.209494590759277, 43.55459213256836, 7.661229133605957, 11.539876937866211, 13.841136932373047, 7.6836018562316895, 9.803584098815918, 97.57599639892578, 12.05272388458252, 7.632177352905273, 13.619303703308105, 8.537130355834961, 11.927653312683105, 12.2246675491333, 8.809345245361328, 11.627182006835938, 9.526654243469238, 14.233187675476074, 11.62244701385498, 58.79878234863281, 7.532905578613281, 16.391521453857422, 10.015725135803223, 8.053321838378906, 7.104892730712891, 14.961706161499023, 8.619656562805176, 10.849471092224121, 13.916534423828125, 8.909649848937988, 31.208776473999023, 6.940392017364502, 7.094511032104492, 8.133481979370117, 10.435072898864746, 10.152830123901367, 12.413495063781738, 7.2537713050842285, 6.450122833251953, 11.192368507385254, 8.942100524902344, 4.699675559997559, 12.313740730285645, 125.51815795898438, 12.43598461151123, 19.431676864624023, 10.207243919372559, 7.279656887054443, 54.94890594482422, 6.60963249206543, 12.085000991821289, 7.8587727546691895, 11.434664726257324, 12.315407752990723, 11.960753440856934, 9.175424575805664, 13.492082595825195, 11.000215530395508, 12.029706001281738, 12.273324012756348, 11.93346118927002, 131.7731475830078, 14.074993133544922, 12.386004447937012, 6.888235092163086, 11.964179039001465, 15.374632835388184, 7.026695251464844, 13.32774829864502, 12.005021095275879, 10.596494674682617, 8.223715782165527, 12.096651077270508, 14.488471984863281, 7.950586795806885, 11.00441837310791, 10.182257652282715, 10.568408966064453, 11.959754943847656, 11.765440940856934, 7.515039920806885, 12.603854179382324, 11.051915168762207, 5.7278828620910645, 124.92913818359375, 6.97782564163208, 12.509150505065918, 9.038175582885742, 12.87226676940918, 9.82371711730957, 68.87975311279297, 7.758638381958008, 9.790884017944336, 10.17648983001709, 12.703986167907715, 14.380447387695312, 41.212154388427734, 9.83743667602539, 7.327874660491943, 12.117881774902344, 18.78571891784668, 6.256680011749268, 16.455059051513672, 6.2428107261657715, 69.91600036621094, 15.710633277893066, 8.539323806762695, 9.379744529724121, 9.377449989318848, 11.551024436950684, 28.02140998840332, 10.767027854919434, 9.924826622009277, 18.79568862915039, 138.07711791992188, 13.044177055358887, 8.973923683166504, 10.322072982788086, 162.64515686035156, 11.61170768737793, 8.656543731689453, 13.21484375, 12.38656997680664, 7.09371280670166, 10.942949295043945, 14.80447769165039, 6.8258056640625, 9.373003959655762, 8.039969444274902, 10.17825984954834, 4.4933648109436035, 10.243800163269043, 9.891965866088867, 8.01807689666748, 8.345907211303711, 12.736761093139648, 11.925952911376953, 10.648653984069824, 8.74402141571045, 7.332059860229492, 11.080114364624023, 10.751019477844238, 10.36120319366455, 10.412834167480469, 8.080814361572266, 9.103134155273438, 5.374403953552246, 9.515564918518066, 13.54983139038086, 10.436921119689941, 10.21725082397461, 16.435373306274414, 9.216620445251465, 7.11768913269043, 8.907283782958984, 9.75093936920166, 11.821206092834473, 9.64370059967041, 9.772671699523926, 125.47123718261719, 12.129541397094727, 11.565688133239746, 8.712538719177246, 10.539806365966797, 13.030142784118652, 7.005171775817871, 7.1676344871521, 9.072322845458984, 7.593445301055908, 5.53422737121582, 6.778730869293213, 4.748504638671875, 10.76831340789795, 7.196353435516357, 9.788043022155762, 13.509886741638184, 7.832637786865234, 7.205541133880615, 9.51665210723877, 10.606400489807129, 151.134521484375, 12.6852445602417, 7.1523871421813965, 6.1322174072265625, 4.76414155960083, 8.974821090698242, 12.353992462158203, 12.967570304870605, 157.3318634033203, 14.956148147583008, 8.152643203735352, 11.168329238891602, 171.2383270263672, 7.994150638580322, 8.479779243469238, 10.888961791992188, 12.19699478149414, 11.67083740234375, 15.139479637145996, 10.587275505065918, 7.123775959014893, 6.6065850257873535, 11.169898986816406, 46.21822738647461, 10.828451156616211, 7.788081169128418, 7.992183208465576, 10.328018188476562, 5.896417617797852, 11.688705444335938, 7.042701721191406, 7.098032474517822, 22.98004913330078, 14.839715957641602, 14.711938858032227, 10.720291137695312, 5.485676288604736, 164.55419921875, 6.784092426300049, 9.935748100280762, 7.121463298797607, 9.864093780517578, 16.2462158203125, 9.935206413269043, 12.749641418457031, 10.955694198608398, 8.06507682800293, 35.163291931152344, 101.37566375732422, 8.56116008758545, 6.355525016784668, 6.965339660644531, 5.818360328674316, 10.844890594482422, 12.734667778015137, 85.9964370727539, 9.488465309143066, 5.897581577301025, 12.929194450378418, 8.06149673461914, 7.483022689819336, 7.097613334655762, 11.786239624023438, 13.628649711608887, 10.480524063110352, 11.926592826843262, 7.958043575286865, 12.470115661621094, 9.32081413269043, 62.68495559692383, 5.844089031219482, 12.587822914123535, 10.82182502746582, 6.909486770629883, 11.285058975219727, 9.661957740783691, 88.56687927246094, 12.061905860900879, 8.658792495727539, 11.201515197753906, 16.491140365600586, 71.48680877685547, 7.666897773742676, 8.94006061553955, 11.317460060119629, 12.599252700805664, 13.644338607788086, 10.726734161376953, 7.774494647979736, 6.806072235107422, 13.20944881439209, 28.64979362487793, 12.38347339630127, 10.507025718688965, 51.07464599609375, 8.944443702697754, 7.25421667098999, 11.582761764526367, 9.979140281677246, 7.12678337097168, 14.311725616455078, 5.699569225311279, 11.937905311584473, 8.373347282409668, 6.877315998077393, 8.338410377502441, 8.544483184814453, 13.106624603271484, 10.716497421264648, 8.144597053527832, 8.012120246887207, 12.31307601928711, 10.842309951782227, 15.305228233337402, 6.290129661560059, 10.429510116577148, 5.9861674308776855, 9.440337181091309, 17.49754524230957, 14.77270221710205, 6.506951332092285, 7.709735870361328, 8.314828872680664, 10.187809944152832, 10.984899520874023, 11.816252708435059, 9.947807312011719, 17.152070999145508, 7.775396823883057, 10.039383888244629, 9.37499713897705, 13.081342697143555, 13.691290855407715, 135.681396484375, 10.002488136291504, 6.128605842590332, 8.923201560974121, 10.880504608154297, 13.574477195739746, 9.993856430053711, 11.595870971679688, 8.42276668548584, 8.92846393585205, 11.373915672302246, 12.182083129882812, 14.329200744628906, 5.643529891967773, 12.980206489562988, 46.11442184448242, 40.17426681518555, 13.033029556274414, 11.405782699584961, 8.294037818908691, 13.021024703979492, 17.109298706054688, 9.660011291503906, 13.644617080688477, 10.841521263122559, 8.425841331481934, 12.955839157104492, 12.722868919372559, 9.628959655761719, 9.013426780700684, 8.520203590393066, 8.089946746826172, 11.803446769714355, 8.239255905151367, 7.636043548583984, 10.917089462280273, 9.215389251708984, 13.15833854675293, 11.010553359985352, 14.912460327148438, 9.945724487304688, 6.101208209991455, 61.45795822143555, 8.458230018615723, 14.452469825744629, 9.706295013427734, 14.661471366882324, 8.40026569366455, 7.079024314880371, 15.446341514587402, 11.066933631896973, 16.363515853881836, 59.80073928833008, 7.82399320602417, 8.38767147064209, 11.568313598632812, 7.891071319580078, 7.937081813812256, 10.509736061096191, 9.836861610412598, 10.385560989379883, 13.854815483093262, 12.7799711227417, 92.85665130615234, 12.05060863494873, 12.276717185974121, 13.018056869506836, 9.58035945892334, 12.259230613708496, 14.39330005645752, 10.978039741516113, 10.054206848144531, 10.75507926940918, 13.847976684570312, 13.570457458496094, 10.374269485473633, 9.278886795043945, 31.909025192260742, 10.939109802246094, 10.728137016296387, 6.1039934158325195, 12.573954582214355, 6.820774555206299, 8.206198692321777, 9.878891944885254, 14.401214599609375, 8.387619972229004, 9.690135955810547, 8.177891731262207, 9.113164901733398, 62.58755874633789, 8.60918140411377, 28.932218551635742, 9.598410606384277, 9.743210792541504, 6.630816459655762, 14.741215705871582, 128.02716064453125, 12.948918342590332, 61.230079650878906, 12.006970405578613, 11.786048889160156, 11.27309513092041, 6.126391410827637, 11.161685943603516, 10.064764022827148, 12.767017364501953, 83.43270874023438, 13.209148406982422, 5.1311774253845215, 7.290121078491211, 7.476589202880859, 7.953354358673096, 12.201285362243652, 13.281624794006348, 15.115241050720215, 12.876931190490723, 7.130945682525635, 13.227556228637695, 14.09790325164795, 7.395371437072754, 8.866726875305176, 19.305227279663086, 10.438390731811523, 10.11269474029541, 8.616761207580566, 7.606306552886963, 6.886057376861572, 8.234325408935547, 6.8270063400268555, 10.779391288757324, 8.487421989440918, 9.224771499633789, 12.237443923950195, 10.294766426086426, 8.718208312988281, 10.41969108581543, 8.308839797973633, 7.517880916595459, 184.64715576171875, 141.6370849609375, 15.653738975524902, 11.710907936096191, 12.796603202819824, 11.154906272888184, 12.216304779052734, 76.59458923339844, 10.520419120788574, 8.70744800567627, 10.455131530761719, 11.129499435424805, 77.33880615234375, 13.46440315246582, 8.521444320678711, 9.738466262817383, 11.188061714172363, 24.04776382446289, 6.3592000007629395, 9.67898941040039, 11.175697326660156, 6.743690490722656, 11.022109031677246, 9.677467346191406, 9.357389450073242, 6.601951599121094, 15.611376762390137, 47.07630920410156, 13.15890884399414, 7.5318708419799805, 10.051185607910156, 9.48550796508789, 12.610442161560059, 14.245682716369629, 7.581748962402344, 12.946938514709473, 13.452126502990723, 99.2339096069336, 82.17808532714844, 11.893221855163574, 9.880685806274414, 13.501319885253906, 9.825427055358887, 7.847043514251709, 14.212804794311523, 7.725376605987549, 12.001526832580566, 15.1658296585083, 10.180276870727539, 6.527350425720215, 9.569868087768555, 5.4096832275390625, 8.112955093383789, 6.980813980102539, 9.612403869628906, 8.456001281738281, 53.46127700805664, 12.515877723693848, 11.510659217834473, 12.799205780029297, 10.200428009033203, 16.803712844848633, 5.74830961227417, 13.291324615478516, 13.479777336120605, 11.291779518127441, 13.121814727783203, 12.711220741271973, 11.259249687194824, 10.458341598510742, 8.68198013305664, 10.119598388671875, 14.9616060256958, 8.720376968383789, 11.585831642150879, 8.27094554901123, 14.1958646774292, 80.07695770263672, 15.75551700592041, 7.416982173919678, 12.361512184143066, 9.692212104797363, 34.40924835205078, 71.39830780029297, 11.175630569458008, 8.642889976501465, 7.511188507080078, 8.317734718322754, 10.131746292114258, 7.022180557250977, 6.282477378845215, 7.93630313873291, 10.10734748840332, 13.599860191345215, 9.172714233398438, 5.179678440093994, 12.420513153076172, 7.554615497589111, 14.138901710510254, 6.4107255935668945, 20.33248519897461, 8.940114974975586, 12.792183876037598, 8.288690567016602, 7.777626037597656, 11.422338485717773, 12.688399314880371, 9.297384262084961, 8.84965705871582, 8.086915969848633, 9.212485313415527, 12.135547637939453, 15.011253356933594, 10.147847175598145, 10.052956581115723, 11.445021629333496, 95.716796875, 10.72482967376709, 9.662261009216309, 54.9752082824707, 15.79900074005127, 13.768576622009277, 5.912846565246582, 8.708810806274414, 8.51169490814209, 13.938284873962402, 9.085822105407715, 8.778464317321777, 9.370527267456055, 7.6729865074157715, 6.205592632293701, 9.722508430480957, 10.387370109558105, 108.50634002685547, 10.443687438964844, 13.958168029785156, 12.369373321533203, 9.123228073120117, 7.600468635559082, 7.988519191741943, 7.300988674163818, 12.043509483337402, 13.961845397949219, 146.5440216064453, 9.455394744873047, 7.832125663757324, 102.45671081542969, 12.491928100585938, 9.728680610656738, 12.13231086730957, 12.949172973632812, 8.981499671936035, 8.51118278503418, 11.626049041748047, 9.088545799255371, 13.06662654876709, 10.653509140014648, 8.960265159606934, 6.224270820617676, 11.250295639038086, 13.502585411071777, 6.116810321807861, 8.271726608276367, 83.8957748413086, 9.45200252532959, 84.13172912597656, 93.97117614746094, 13.951732635498047, 10.67402458190918, 11.000316619873047, 8.357451438903809, 8.190590858459473, 8.78583812713623, 14.125652313232422, 9.214936256408691, 7.564579963684082, 13.265812873840332, 10.547629356384277, 9.69953441619873, 5.788376331329346, 6.389457702636719, 130.42254638671875, 8.836603164672852, 9.187278747558594, 9.688385963439941, 15.62948989868164, 6.428660869598389, 8.89063549041748, 14.179764747619629, 11.08752727508545, 12.399700164794922, 4.135590076446533, 9.919371604919434, 45.27794647216797, 47.68242263793945, 8.802275657653809, 99.4503402709961, 6.251859664916992, 13.080254554748535, 8.611331939697266, 10.966951370239258, 5.931273460388184, 10.130364418029785, 7.350490570068359, 10.117898941040039, 15.364785194396973, 7.805467128753662, 5.686593532562256, 34.92665481567383, 6.118268013000488, 10.244970321655273, 8.390798568725586, 7.655313014984131, 9.978180885314941, 8.600232124328613, 8.646315574645996, 91.93183135986328, 10.880681991577148, 10.305390357971191, 12.950867652893066, 10.215865135192871, 8.898987770080566, 12.027520179748535, 125.43727111816406, 5.365444660186768, 5.298332214355469, 15.258894920349121, 9.268908500671387, 52.57343292236328, 6.868773460388184, 15.785597801208496, 9.301637649536133, 10.067229270935059, 10.835108757019043, 55.42573547363281, 16.310325622558594, 12.904498100280762, 14.252739906311035, 9.159798622131348, 9.056814193725586, 7.149116516113281, 9.146717071533203, 20.07988739013672, 168.134765625, 10.98067569732666, 7.855212688446045, 12.905230522155762, 11.280523300170898, 8.083879470825195, 10.125384330749512, 5.375886917114258, 9.272320747375488, 10.879171371459961, 11.741787910461426, 9.254493713378906, 6.159470558166504, 10.722097396850586, 6.461339473724365, 12.834364891052246, 9.381175994873047, 14.031510353088379, 9.972941398620605, 16.73098373413086, 9.890151023864746, 12.793328285217285, 13.789767265319824, 6.020062446594238, 10.677660942077637, 8.994258880615234, 15.041912078857422, 11.215167999267578, 38.0975227355957, 9.07327938079834, 6.591241836547852, 12.276583671569824, 11.935400009155273, 13.27373218536377, 82.61239624023438, 13.898408889770508, 11.960620880126953, 81.5862808227539, 10.586915016174316, 11.595649719238281, 8.80179500579834, 11.000468254089355, 52.24696731567383, 12.604500770568848, 5.332718849182129, 9.675524711608887, 8.713143348693848, 11.3366060256958, 6.313329219818115, 14.445651054382324, 7.67373514175415, 10.44430923461914, 12.971414566040039, 13.869746208190918, 14.126996994018555, 7.3622026443481445, 84.91029357910156, 9.129143714904785, 8.681785583496094, 13.322566032409668, 7.694470405578613, 12.078047752380371, 113.96116638183594, 6.637872695922852, 6.499022960662842, 81.00729370117188, 7.152814865112305, 11.768744468688965, 14.450057983398438, 7.7425150871276855, 14.239595413208008, 62.625701904296875, 11.27009105682373, 8.584609031677246, 13.49083137512207, 8.494190216064453, 15.682951927185059, 8.662631034851074, 11.970847129821777, 8.90447998046875, 7.966714382171631, 14.571756362915039, 13.62926197052002, 10.11423397064209, 8.592534065246582, 5.107819080352783, 166.83189392089844, 13.3112211227417, 7.167262554168701, 12.15262222290039, 8.291545867919922, 8.620311737060547, 9.042815208435059, 116.92237854003906, 16.3757266998291, 8.81736946105957, 8.758439064025879, 14.041220664978027, 12.456280708312988, 63.16281509399414, 5.842182636260986, 7.073546409606934, 16.0301456451416, 97.8067398071289, 9.746829986572266, 10.763726234436035, 30.47757339477539, 14.635505676269531, 9.522488594055176, 10.74715805053711, 10.739845275878906, 7.533633232116699, 10.573914527893066, 12.088793754577637, 10.252466201782227, 13.410429954528809, 10.345157623291016, 9.157394409179688, 99.20038604736328, 9.221344947814941, 7.327656269073486, 13.485997200012207, 13.019123077392578, 6.36203145980835, 11.611146926879883, 13.00656509399414, 9.89769172668457, 10.421979904174805, 11.96355152130127, 16.13709259033203, 8.60387134552002, 8.254701614379883, 6.956965923309326, 100.84805297851562, 10.077869415283203, 9.866966247558594, 7.170833587646484, 31.83701515197754, 7.234972953796387, 6.860209941864014, 10.31520938873291, 9.490654945373535, 6.280998706817627, 14.133574485778809, 10.590038299560547, 6.924856185913086, 55.47046661376953, 9.431743621826172, 13.691912651062012, 6.148157119750977, 65.47508239746094, 8.57532024383545, 7.370568752288818, 11.874297142028809, 12.162102699279785, 33.708927154541016, 11.349466323852539, 14.818262100219727, 8.550101280212402, 41.32939147949219, 9.623185157775879, 102.91543579101562, 9.002058982849121, 5.228865623474121, 5.852513790130615, 86.69799041748047, 7.075172424316406, 7.91940450668335, 12.33603572845459, 10.781583786010742, 10.490886688232422, 10.218883514404297, 7.501969814300537, 7.634880065917969, 7.958436489105225, 12.829568862915039, 9.803378105163574, 11.216025352478027, 10.929434776306152, 8.870875358581543, 10.48332405090332, 9.563210487365723, 9.84194278717041, 11.135940551757812, 11.722764015197754, 184.006591796875, 143.88479614257812, 18.895368576049805, 12.507081985473633, 12.039790153503418, 7.861772060394287, 8.98457145690918, 7.59069299697876, 8.275012016296387, 14.08084487915039, 8.192880630493164, 5.311314582824707, 11.031265258789062, 11.139470100402832, 9.958640098571777, 12.313224792480469, 8.809569358825684, 151.51014709472656, 11.049409866333008, 8.2852144241333, 13.652324676513672, 11.265145301818848, 12.355884552001953, 35.86441421508789, 9.698943138122559, 8.195581436157227, 5.855514049530029, 8.369463920593262, 6.990524768829346, 12.675151824951172, 10.87488842010498, 9.752778053283691, 12.435619354248047, 116.0216064453125, 134.84950256347656, 8.930394172668457, 10.806048393249512, 9.47705078125, 11.096943855285645, 10.852831840515137, 14.769763946533203, 15.679210662841797, 11.662224769592285, 7.438094139099121, 12.869140625, 15.573598861694336, 9.681045532226562, 10.917802810668945, 8.39182186126709, 10.939546585083008, 120.94441223144531, 32.65741729736328, 8.563232421875, 10.184385299682617, 10.808767318725586, 7.057926654815674, 7.8637871742248535, 14.079039573669434, 11.452043533325195, 17.475317001342773, 8.148015975952148, 7.051718711853027, 9.580829620361328, 11.036211013793945, 10.413451194763184, 10.977067947387695, 11.385698318481445, 6.3980021476745605, 12.146440505981445, 7.660039901733398, 9.602107048034668, 13.540433883666992, 65.25055694580078, 10.125898361206055, 7.348972797393799, 12.437222480773926, 7.720070838928223, 12.689254760742188, 8.86545181274414, 11.669523239135742, 10.847835540771484, 6.897218227386475, 14.713400840759277, 5.706949234008789, 12.304059028625488, 10.299908638000488, 32.360836029052734, 12.06911563873291, 8.575831413269043, 9.265976905822754, 8.201340675354004, 9.049820899963379, 11.11157512664795, 14.933008193969727, 8.296216011047363, 5.687321662902832, 139.4875946044922, 7.054847717285156, 9.785991668701172, 9.58035945892334, 104.02853393554688, 150.2911376953125, 9.479077339172363, 6.675470352172852, 9.799100875854492, 15.446053504943848, 10.330340385437012, 10.636968612670898, 7.8359904289245605, 9.961202621459961, 9.36411190032959, 14.18304443359375, 12.847265243530273, 10.24740982055664, 5.971119403839111, 9.589256286621094, 8.924275398254395, 62.453922271728516, 26.780784606933594, 8.944860458374023, 11.70447063446045, 7.278271675109863, 14.374975204467773, 10.555154800415039, 6.932608127593994, 9.737407684326172, 6.3713059425354, 177.14244079589844, 11.753536224365234, 12.611215591430664, 14.453601837158203, 6.324678897857666, 11.44094181060791, 11.751428604125977, 11.385708808898926, 14.337763786315918, 8.390739440917969, 14.810944557189941, 14.05929946899414, 8.410384178161621, 8.504417419433594, 6.144503116607666, 8.594513893127441, 8.487384796142578, 6.394866943359375, 10.290647506713867, 9.755087852478027, 11.303696632385254, 9.513947486877441, 13.469654083251953, 8.746536254882812, 9.690265655517578, 11.689111709594727, 115.74011993408203, 13.25908374786377, 8.346940994262695, 9.502741813659668, 7.125422477722168, 5.4025797843933105, 6.150968551635742, 14.324934959411621, 9.878321647644043, 6.232572555541992, 9.727468490600586, 8.77363109588623, 7.726089954376221, 10.941521644592285, 8.811673164367676, 12.926952362060547, 14.627755165100098, 8.802165985107422, 10.707025527954102, 9.647273063659668, 8.537854194641113, 11.664072036743164, 11.817273139953613, 13.207528114318848, 12.771533012390137, 15.137580871582031, 10.274266242980957, 83.78324127197266, 6.928097724914551, 8.98684024810791, 11.59355354309082, 9.01375675201416, 13.011137962341309, 8.170097351074219, 14.437063217163086, 95.97988891601562, 8.68386459350586, 13.435153007507324, 5.824111461639404, 13.600372314453125, 36.678382873535156, 12.607892990112305, 15.315923690795898, 9.360669136047363, 9.627144813537598, 169.39988708496094, 11.584616661071777, 11.336258888244629, 12.549910545349121, 28.14179801940918, 12.659444808959961, 10.770342826843262, 9.008572578430176, 9.84736156463623, 8.566658973693848, 10.575913429260254, 11.488082885742188, 11.850885391235352, 9.446471214294434, 8.750503540039062, 8.20938491821289, 10.589385032653809, 7.640331268310547, 7.734945297241211, 33.6673469543457, 11.938060760498047, 122.92508697509766, 14.499114036560059, 9.758487701416016, 7.02360200881958, 12.954343795776367, 7.181325912475586, 61.79834747314453, 11.307148933410645, 7.701659679412842, 7.731200218200684, 48.83842086791992, 6.360012054443359, 29.779712677001953, 179.6275634765625, 11.450660705566406, 11.413419723510742, 7.66916036605835, 12.443758964538574, 12.225529670715332, 14.884353637695312, 7.536264419555664, 8.542668342590332, 8.122905731201172, 7.60122537612915, 8.816506385803223, 13.175515174865723, 14.131858825683594, 7.098019599914551, 13.09226131439209, 13.423056602478027, 12.339883804321289, 8.000221252441406, 8.166523933410645, 9.369150161743164, 10.920666694641113, 9.667953491210938, 10.26478099822998, 7.608031272888184, 9.061347007751465, 131.97662353515625, 9.452729225158691, 9.488876342773438, 12.09018611907959, 12.273371696472168, 6.715031623840332, 9.2130708694458, 36.054054260253906, 8.087966918945312, 39.741180419921875, 7.563966751098633, 7.230350017547607, 7.709529399871826, 5.70667028427124, 9.170527458190918, 6.337283611297607, 10.70485782623291, 5.667797565460205, 8.2752685546875, 8.638166427612305, 9.567564964294434, 8.54906940460205, 78.94154357910156, 7.445744514465332, 10.615072250366211, 12.577038764953613, 8.414030075073242, 6.987974643707275, 7.357454299926758, 6.351019382476807, 12.165016174316406, 10.70568561553955, 7.441814422607422, 10.283923149108887, 8.272430419921875, 32.02134704589844, 9.771450996398926, 12.125767707824707, 10.315912246704102, 11.866976737976074, 85.17827606201172, 7.33848762512207, 12.960702896118164, 7.250901222229004, 8.827888488769531, 9.790120124816895, 6.382519721984863, 13.51625919342041, 9.636122703552246, 15.501376152038574, 51.65472412109375, 11.168225288391113, 30.198049545288086, 8.766824722290039, 15.81344985961914, 16.258071899414062, 10.479714393615723, 6.329509258270264, 144.1278076171875, 9.635931015014648, 10.199342727661133, 10.905943870544434, 10.932564735412598, 14.963078498840332, 136.46060180664062, 10.792553901672363, 11.133785247802734, 10.134986877441406, 14.292720794677734, 9.6896390914917, 10.682938575744629, 9.255425453186035, 14.827295303344727, 6.9998250007629395, 10.762107849121094, 11.809478759765625, 11.91707992553711, 8.609121322631836, 8.77408218383789, 7.2435736656188965, 12.936749458312988, 9.86866569519043, 8.340821266174316, 14.286107063293457, 7.206716060638428, 7.8157639503479, 7.323019504547119, 13.650534629821777, 20.944629669189453, 15.710335731506348, 13.571733474731445, 7.849580764770508, 50.525726318359375, 11.12005615234375, 17.256343841552734, 8.605890274047852, 10.333585739135742, 12.617764472961426, 8.46265983581543, 9.224575996398926, 10.880851745605469, 10.314491271972656, 6.421219348907471, 15.2290678024292, 12.104241371154785, 8.024632453918457, 7.745706558227539, 86.68299102783203, 103.66110229492188, 9.21570110321045, 9.853001594543457, 80.94850158691406, 9.665407180786133, 14.800790786743164, 7.364584922790527, 5.502750396728516, 9.39063835144043, 14.321104049682617, 7.766106605529785, 111.89472961425781, 8.215228080749512, 7.775803089141846, 9.2493314743042, 12.814549446105957, 65.46758270263672, 7.725686073303223, 11.640156745910645, 11.407639503479004, 9.114760398864746, 5.930400848388672, 10.752026557922363, 12.583166122436523, 6.075570583343506, 11.914289474487305, 10.492727279663086, 16.51800537109375, 9.02777099609375, 123.9871597290039, 111.49382019042969, 9.14414119720459, 66.61713409423828, 9.508708000183105, 7.9064154624938965, 6.946544647216797, 15.146379470825195, 9.505658149719238, 7.737450122833252, 5.889995098114014, 8.952448844909668, 5.769715785980225, 14.782773971557617, 7.389853000640869, 8.025039672851562, 5.585808753967285, 11.6508150100708, 15.21822738647461, 8.287445068359375, 11.99267864227295, 13.165090560913086, 12.542529106140137, 9.558660507202148, 9.757770538330078, 22.823150634765625, 8.721837043762207, 13.546914100646973, 10.231741905212402, 7.488245010375977, 7.9360527992248535, 12.731461524963379, 10.810888290405273, 10.847280502319336, 6.522859573364258, 12.754903793334961, 13.332948684692383, 11.29499626159668, 12.781105041503906, 8.23114013671875, 11.214980125427246, 10.651893615722656, 9.09118366241455, 6.889470100402832, 6.777736663818359, 11.3765287399292, 9.817466735839844, 6.973762035369873, 9.752911567687988, 11.629106521606445, 9.150697708129883, 8.659717559814453, 8.83010196685791, 10.791232109069824, 11.3357515335083, 12.871551513671875, 9.476003646850586, 13.431339263916016, 11.02620792388916, 96.91594696044922, 11.380829811096191, 17.382205963134766, 14.57026195526123, 9.688186645507812, 13.603473663330078, 10.741388320922852, 22.295137405395508, 85.68809509277344, 12.376687049865723, 5.41035270690918, 8.90788459777832, 58.7234992980957, 9.981385231018066, 11.129622459411621, 7.35018253326416, 6.382748603820801, 10.175714492797852, 38.145206451416016, 7.352074146270752, 6.967290878295898, 5.891641139984131, 10.916298866271973, 144.3795166015625, 10.446087837219238, 10.534924507141113, 12.864205360412598, 13.238638877868652, 14.415462493896484, 13.913863182067871, 14.47420597076416, 12.393916130065918, 6.900791168212891, 11.536154747009277, 167.6299285888672, 9.103278160095215, 12.3056058883667, 99.28804779052734, 7.674367904663086, 14.407424926757812, 13.953489303588867, 33.88032531738281, 9.076512336730957, 7.176352024078369, 7.138691425323486, 11.701003074645996, 11.25541877746582, 11.29819393157959, 7.236807346343994, 10.210348129272461, 5.923393726348877, 11.401826858520508, 47.65364074707031, 144.91500854492188, 7.88155460357666, 8.594030380249023, 7.706475257873535, 6.851680755615234, 15.8389892578125, 6.375034809112549, 6.652584075927734, 11.897642135620117, 9.575647354125977, 12.895008087158203, 9.42745304107666, 9.202401161193848, 10.531691551208496, 9.45225715637207, 8.19002628326416, 58.21119689941406, 12.200345993041992, 12.681177139282227, 8.773879051208496, 12.260782241821289, 7.817621231079102, 52.79658508300781, 9.602380752563477, 8.778944969177246, 7.268777847290039, 11.1178617477417, 8.227170944213867, 6.809329509735107, 16.435636520385742, 9.260488510131836, 162.55723571777344]


In [25]:
sum(disances_random_graphs) / len(disances_random_graphs)

18.113435906228567

In [26]:
cosine_similarities_random_graphs = [0.3590472638607025, 0.13720102608203888, 0.49049004912376404, -0.35519272089004517, 0.07015152275562286, 0.5953618288040161, 0.045064907521009445, 0.32656168937683105, 0.3754318058490753, 0.12288619577884674, 0.6552219390869141, -0.03262774646282196, 0.15948449075222015, -0.1620180904865265, -0.6225457191467285, 0.39673835039138794, 0.023668557405471802, 0.3971911072731018, -0.7147612571716309, -0.11939266324043274, -0.30487000942230225, -0.12869924306869507, 0.5296878814697266, -0.7796765565872192, -0.055763546377420425, 0.004641138017177582, 0.5113300681114197, 0.6503061652183533, 0.5211899280548096, 0.2972983717918396, 0.5250338315963745, 0.27694517374038696, 0.15140166878700256, -0.23348791897296906, 0.44664299488067627, 0.8924641609191895, 0.05029173195362091, 0.4025992155075073, -0.3889516592025757, 0.11499622464179993, -0.6873579025268555, 0.03911487013101578, 0.21598049998283386, 0.03902445733547211, 0.08083224296569824, 0.6132083535194397, 0.18596503138542175, 0.40323224663734436, 0.34621483087539673, -0.2719583511352539, 0.4072636067867279, 0.78242027759552, -0.03456474840641022, -0.18386507034301758, 0.32207465171813965, 0.09474325180053711, 0.10312467068433762, 0.45096296072006226, 0.6814846396446228, 0.042642828077077866, 0.16193807125091553, 0.4844445586204529, 0.7079476118087769, -0.09372227638959885, 0.7369586229324341, -0.3841826021671295, 0.20592084527015686, 0.2528344988822937, 0.10650020837783813, -0.08725021779537201, 0.4296379089355469, 0.03442871570587158, -0.06325513124465942, 0.6836147308349609, 0.4431939125061035, 0.24999764561653137, 0.5271698236465454, -0.49814245104789734, 0.05366379767656326, 0.1990738809108734, -0.16942855715751648, -0.6268934011459351, -0.6677452325820923, 0.15732000768184662, 0.11426079273223877, 0.17786511778831482, -0.5360944271087646, 0.10581576824188232, 0.44589900970458984, 0.5266794562339783, 0.7327489852905273, -0.16594454646110535, -0.5116084814071655, -0.4869481921195984, 0.21549658477306366, -0.03998900204896927, -0.10732848197221756, -0.39691323041915894, -0.2918597459793091, 0.15437781810760498, 0.4826802611351013, -0.12347190827131271, 0.1357279121875763, 0.5165252089500427, -0.04016980901360512, -0.2581014931201935, -0.6182479858398438, -0.253617525100708, 0.028782546520233154, -0.21289266645908356, -0.022432168945670128, -0.5218465328216553, 0.15749654173851013, 0.23433083295822144, 0.14740437269210815, -0.23679369688034058, -0.3649860620498657, 0.30201607942581177, -0.29720962047576904, 0.6337924003601074, 0.02237747609615326, -0.4280933737754822, -0.05947583168745041, -0.4438943862915039, 0.23452939093112946, 0.5867527723312378, 0.586610734462738, -0.19159355759620667, -0.3909931480884552, 0.23885977268218994, 0.5649368166923523, 0.5257251262664795, 0.021427027881145477, 0.6603782176971436, 0.5062952637672424, -0.21635757386684418, -0.87409508228302, -0.15624666213989258, -0.01418173499405384, -0.45224496722221375, 0.6450602412223816, 0.44321876764297485, 0.42612579464912415, 0.8583856821060181, -0.542037844657898, -0.5618491172790527, -0.382797509431839, -0.30935657024383545, 0.8990582823753357, -0.37114089727401733, 0.7801336050033569, -0.28255000710487366, -0.16466522216796875, 0.3576527535915375, -0.2962695360183716, -0.20565102994441986, 0.7063698768615723, 0.5089045763015747, -0.08226041495800018, 0.8223099708557129, -0.2799108028411865, 0.14325043559074402, 0.3808381259441376, 0.2549895644187927, 0.04710422828793526, 0.4660639464855194, -0.08794111013412476, 0.1593799591064453, 0.1111980676651001, 0.21698619425296783, -0.3491089940071106, -0.11371737718582153, 0.06899552047252655, 0.3879847824573517, -0.5743595361709595, -0.7473585605621338, 0.4950745105743408, -0.2221752405166626, -0.04133123159408569, -0.4090414047241211, 0.33307623863220215, 0.28378617763519287, -0.07339109480381012, 0.7708094120025635, 0.4035734534263611, 0.08743010461330414, -0.4136442542076111, -0.08835543692111969, 0.4637483060359955, 0.590149998664856, -0.30104756355285645, 0.009781919419765472, -0.26300716400146484, -0.0826009213924408, -0.005376897752285004, -0.33747851848602295, 0.007638797163963318, -0.3765410780906677, 0.2539157271385193, 0.44877129793167114, -0.6469606161117554, 0.3852216601371765, -0.4921208620071411, 0.707302451133728, -0.578274130821228, 0.001115269958972931, 0.07552662491798401, 0.5760418176651001, -0.5571152567863464, -0.015377655625343323, 0.07150667160749435, 0.08127033710479736, -0.34750354290008545, 0.25126826763153076, -0.3779413104057312, 0.4748568534851074, 0.4162899851799011, 0.5961226224899292, 0.41192299127578735, -0.3930895924568176, 0.02162931114435196, 0.29126450419425964, 0.4967693090438843, -0.5664835572242737, -0.6049929857254028, -0.5152214765548706, -0.18179777264595032, -0.06638652086257935, -0.264497309923172, 0.716770350933075, 0.546894907951355, 0.19438394904136658, -0.6208703517913818, 0.7005776166915894, 0.3351346254348755, 0.7312150001525879, 0.23833796381950378, -0.21479225158691406, 0.8443260192871094, -0.5103722810745239, 0.08625143766403198, -0.07623602449893951, -0.19186459481716156, 0.16584709286689758, 0.0646573156118393, 0.05737365037202835, 0.037071093916893005, 0.07123646140098572, -0.020330622792243958, -0.1330464631319046, 0.2051803171634674, 0.5191393494606018, 0.4660188555717468, -0.4859832227230072, 0.7232121229171753, -0.7582805156707764, 0.6020872592926025, 0.810238242149353, 0.8383642435073853, -0.061902955174446106, -0.02498854696750641, 0.006243772804737091, -0.4616231322288513, -0.04778585582971573, 0.36689311265945435, -0.3769039213657379, -0.1362074315547943, -0.23123811185359955, -0.6934296488761902, 0.6373510360717773, -0.19010457396507263, 0.5698117613792419, 0.5511085987091064, 0.6987062692642212, 0.4494832754135132, 0.40459775924682617, 0.7277647256851196, 0.13787716627120972, 0.655146598815918, 0.08333125710487366, -0.4644351005554199, -0.16230031847953796, 0.4334471821784973, -0.173354834318161, 0.6228805780410767, -0.5379510521888733, -0.034471966326236725, -0.4243587553501129, 0.669847846031189, -0.4774104952812195, -0.0017305463552474976, -0.06257236003875732, -0.28051137924194336, -0.432325154542923, 0.5724576711654663, 0.18630407750606537, 0.4627953767776489, -0.19685685634613037, 0.5024145841598511, -0.19885262846946716, 0.038093239068984985, 0.42693835496902466, 0.16042569279670715, -0.05345362052321434, 0.5208486318588257, 0.6399795413017273, -0.5236418843269348, -0.2065754383802414, -0.49085068702697754, -0.04842259734869003, 0.223471537232399, 0.7524282336235046, -0.1260124146938324, 0.242570698261261, 0.025163637474179268, 0.7295138239860535, -0.2967137396335602, -0.330022931098938, 0.3620659112930298, -0.017064236104488373, 0.5606246590614319, 0.2965151071548462, 0.019300824031233788, 0.19790570437908173, 0.4643741846084595, 0.32256728410720825, -0.05686573684215546, -0.025790315121412277, 0.3928918242454529, 0.011598322540521622, 0.6134704351425171, 0.7408713698387146, 0.810987114906311, 0.3581083416938782, -0.10879908502101898, 0.7114669680595398, 0.783940315246582, 0.36845117807388306, 0.17418354749679565, 0.527015745639801, 0.10900208353996277, -0.028976961970329285, -0.23772892355918884, 0.6190474629402161, 0.3276344835758209, -0.03301338851451874, 0.46253877878189087, -0.5109919905662537, -0.254352331161499, 0.7477349638938904, 0.21333518624305725, 0.19848591089248657, -0.33130836486816406, 0.03766787052154541, 0.035069651901721954, -0.19682177901268005, 0.5689617991447449, -0.6417191624641418, -0.4598199725151062, 0.5826443433761597, 0.009648777544498444, 0.06465905159711838, -0.10552258789539337, -0.2921830117702484, -0.6874785423278809, 0.6411932706832886, -0.46128618717193604, -0.43604525923728943, 0.6963546872138977, -0.19356614351272583, -0.41389235854148865, 0.33912038803100586, 0.4657384753227234, -0.09110648930072784, 0.47773826122283936, -0.01966775953769684, -0.23570075631141663, -0.28401052951812744, -0.3126375079154968, 0.5640482306480408, -0.39600712060928345, -0.14712540805339813, -0.20117071270942688, 0.7485837340354919, 0.4686394929885864, 0.428252249956131, -0.4113532304763794, -0.08113051950931549, 0.5285941362380981, -0.36752229928970337, 0.5315796136856079, 0.23333248496055603, 0.14002341032028198, -0.39254140853881836, -0.10887129604816437, -0.04528724402189255, 0.44329991936683655, -0.34969785809516907, -0.04092240333557129, 0.44733917713165283, -0.03904309868812561, 0.13338948786258698, 0.15329472720623016, -0.6317164897918701, -0.22496174275875092, 0.11154236644506454, -0.23291930556297302, 0.07541094720363617, 0.19428585469722748, 0.37913453578948975, -0.6374394297599792, 0.39313429594039917, 0.7713238000869751, 0.7903839945793152, 0.3969971537590027, 0.41972625255584717, -0.25846874713897705, -0.6258280873298645, 0.6380620002746582, -0.7280568480491638, 0.13719569146633148, 0.3189743757247925, -0.0018855370581150055, -0.1054629310965538, -0.5607202053070068, -0.3712296485900879, -0.7448423504829407, 0.6409910321235657, 0.8127986192703247, 0.6417707204818726, 0.5787123441696167, 0.029547326266765594, 0.02618280053138733, -0.020546510815620422, 0.18146349489688873, -0.3907545804977417, -0.1366959661245346, -0.209528848528862, 0.17166346311569214, -0.0650116354227066, -0.7546854019165039, 0.7732797265052795, -0.0059251561760902405, 0.3564617335796356, -0.08940758556127548, -0.7871929407119751, 0.4909040331840515, 0.36880818009376526, 0.20538683235645294, 0.0758218914270401, -0.39749374985694885, 0.22351562976837158, 0.4850762188434601, -0.4219800531864166, 0.09438791871070862, 0.5996130704879761, -0.1286495476961136, -0.0745704174041748, 0.24645593762397766, 0.7252523899078369, -0.11996869742870331, -0.46428465843200684, -0.3976874351501465, 0.6176719069480896, 0.017057429999113083, 0.3760031461715698, -0.5959820747375488, 0.550857663154602, 0.257310152053833, 0.5154210329055786, -0.4186103641986847, -0.018600791692733765, 0.4020099937915802, 0.21256187558174133, -0.21818159520626068, 0.5999826192855835, 0.6261706352233887, 0.31748801469802856, 0.11129467934370041, 0.5215396881103516, -0.3691290318965912, -0.09218941628932953, -0.613659143447876, 0.6796177625656128, -0.3215240240097046, 0.5627284646034241, -0.7953733801841736, 0.30766671895980835, 0.18198874592781067, -0.4837762117385864, 0.17080876231193542, -0.46014657616615295, 0.28555727005004883, 0.22480739653110504, 0.3060436248779297, 0.1908136010169983, 0.5117356181144714, 0.3265570402145386, 0.4891924262046814, 0.2064858078956604, -0.4351425766944885, -0.09846848994493484, 0.45689475536346436, 0.5246871113777161, 0.7843397855758667, -0.0032777339220046997, 0.25290215015411377, -0.2490716427564621, 0.6927778720855713, 0.08639366924762726, 0.5071450471878052, -0.4744161069393158, -0.1794915497303009, -0.04619191586971283, 0.2822415232658386, 0.8224378824234009, 0.5754246711730957, 0.048295579850673676, -0.22052182257175446, -0.19744139909744263, -0.11097046732902527, -0.3250613510608673, 0.08140750229358673, 0.17641830444335938, 0.4850655496120453, 0.602455735206604, 0.10020527243614197, -0.3475865125656128, 0.5612764358520508, -0.29894280433654785, 0.09373775869607925, 0.3871285319328308, 0.21409009397029877, -0.2346983551979065, 0.011090993881225586, 0.6532394886016846, -0.2105078101158142, 0.23953786492347717, 0.21217089891433716, 0.7427617311477661, -0.3901260495185852, 0.3901382088661194, 0.05232501029968262, 0.20268285274505615, -0.13063457608222961, 0.5988485813140869, 0.671680212020874, -0.16439500451087952, 0.2660314440727234, 0.32505565881729126, 0.6112149953842163, 0.27559298276901245, 0.3242512047290802, 0.0021292641758918762, -0.410541832447052, 0.4977525770664215, 0.5488056540489197, -0.6783261895179749, -0.37907081842422485, 0.7174962759017944, 0.3952675461769104, -0.06465168297290802, -0.47880643606185913, 0.335752934217453, 0.35866811871528625, -0.26487401127815247, -0.7394336462020874, -0.5174530744552612, -0.013936586678028107, 0.6284382343292236, -0.3949151635169983, -0.21286870539188385, -0.25560981035232544, 0.826911211013794, 0.015830427408218384, 0.531943678855896, 0.629432201385498, 0.3530736565589905, 0.13810111582279205, -0.5548278093338013, 0.005494376644492149, -0.07830850780010223, 0.3274881839752197, 0.378665566444397, -0.4831233322620392, -0.006408538669347763, -0.17336592078208923, -0.0936976745724678, 0.5444386005401611, 0.19506295025348663, 0.7503232955932617, -0.3800201416015625, 0.7942348718643188, -0.6252769231796265, -0.36544615030288696, 0.18844282627105713, 0.30192995071411133, 0.055169448256492615, 0.351835161447525, -0.05061155930161476, 0.38609734177589417, 0.07308577001094818, -0.31896311044692993, -0.5032485127449036, 0.41632217168807983, -0.3079718351364136, 0.559798538684845, 0.47593075037002563, 0.885873556137085, -0.5903253555297852, 0.38447701930999756, 0.474281907081604, -0.3694422245025635, -0.6008626818656921, -0.2912704646587372, -0.06733755767345428, 0.14100533723831177, -0.3903421461582184, 0.3626747131347656, 0.7484747767448425, 0.0217939130961895, 0.7154223918914795, -0.1273554265499115, -0.5087443590164185, 0.411382794380188, 0.2326059639453888, -0.08281435072422028, -0.4240037798881531, -0.4105371832847595, -0.13867786526679993, 0.15378110110759735, 0.21959033608436584, 0.22129064798355103, 0.4984249174594879, 0.3986286520957947, -0.16826459765434265, 0.28683707118034363, -0.08071596920490265, 0.13081838190555573, -0.24118134379386902, 0.44265347719192505, -0.367467999458313, 0.3669346570968628, 0.13617819547653198, -0.47813016176223755, 0.038964129984378815, 0.8487460613250732, 0.18382596969604492, 0.004860147833824158, -0.3940671682357788, 0.3329448699951172, 0.13300961256027222, -0.2245037853717804, -0.3600127100944519, 0.4954259693622589, 0.04460453242063522, 0.10427403450012207, -0.110776886343956, 0.5533558130264282, 0.7587883472442627, 0.10803420841693878, -0.18894538283348083, 0.30450356006622314, 0.34790849685668945, 0.46045511960983276, 0.5571346879005432, -0.6712466478347778, 0.5975773334503174, -0.15575623512268066, 0.6709240674972534, -0.3785274922847748, 0.5736167430877686, -0.7907785773277283, 0.4599771201610565, -0.08202396333217621, 0.570603609085083, 0.11288955807685852, -0.18208356201648712, 0.3390146493911743, 0.14277982711791992, -0.6266802549362183, 0.6806807518005371, -0.42821717262268066, 0.5160142183303833, -0.3703101873397827, -0.5490259528160095, 0.4468204379081726, 0.3083255887031555, 0.17527899146080017, 0.03286600857973099, 0.46809837222099304, -0.49463918805122375, 0.31605789065361023, -0.48848724365234375, -0.07658981531858444, 0.2554338872432709, -0.39916861057281494, 0.060648806393146515, -0.3200019598007202, -0.3629189729690552, 0.8689537644386292, 0.5610668063163757, -0.673309326171875, 0.07942070811986923, 0.34590771794319153, 0.6063868999481201, -0.16114604473114014, 0.32042196393013, -0.28642016649246216, -0.43513163924217224, 0.25505557656288147, 0.036301277577877045, 0.571260929107666, 0.650796115398407, 0.4402417540550232, -0.06088581681251526, 0.24381577968597412, -0.25224927067756653, 0.6001291275024414, 0.658618688583374, -0.29106393456459045, 0.29727596044540405, 0.8329741954803467, -0.1690458059310913, 0.21079206466674805, -0.23457719385623932, 0.5160418748855591, 0.2999662756919861, 0.5540187954902649, -0.5966544151306152, 0.6234704256057739, -0.07635564357042313, 0.4462915360927582, -0.4744454622268677, -0.47815239429473877, -0.4400642514228821, 0.09603044390678406, -0.4760267734527588, 0.058015599846839905, -0.13676303625106812, -0.2635210156440735, -0.1314612627029419, -0.9002115726470947, 0.8119803667068481, -0.41432443261146545, 0.5896968841552734, -0.246152862906456, -0.6853290796279907, 0.5916849970817566, -0.3675321340560913, -0.39557015895843506, -0.1617366075515747, 0.4187871813774109, -0.14118467271327972, -0.3572154939174652, 0.4933964014053345, -0.09769415855407715, -0.09562669694423676, -0.18481740355491638, -0.36580437421798706, -0.1426408737897873, 0.47706934809684753, -0.3675755262374878, 0.04527774453163147, 0.7579488754272461, -0.7507505416870117, 0.5717121362686157, -0.17172208428382874, 0.09328854084014893, -0.5115001201629639, 0.05923301354050636, 0.6285638213157654, 0.5683607459068298, 0.1125854030251503, 0.08138088136911392, 0.2133043110370636, -0.5816224217414856, 0.44167786836624146, -0.03916575014591217, 0.51570063829422, -0.24751827120780945, -0.31631869077682495, 0.6727880835533142, -0.7380918264389038, 0.6761863231658936, -0.7664259672164917, 0.028352811932563782, 0.3187442421913147, 0.08239859342575073, 0.23769806325435638, -0.2920963764190674, -0.002961866557598114, -0.05992893502116203, -0.0038917697966098785, -0.6617507934570312, 0.42749321460723877, -0.517100989818573, 0.21317632496356964, 0.07592752575874329, 0.13687187433242798, 0.03320905566215515, 0.2420322597026825, -0.3061176538467407, -0.3351040184497833, 0.5990710854530334, -0.10872412472963333, -0.5600749254226685, 0.6263960599899292, 0.009329169988632202, 0.42167389392852783, -0.10476615279912949, 0.8492625951766968, -0.00972273200750351, -0.09085170179605484, 0.3957056701183319, 0.3566957116127014, -0.3396482467651367, -0.34958770871162415, -0.13923108577728271, 0.3272312879562378, 0.5522323846817017, 0.11679539084434509, 0.0041680000722408295, -0.1295505315065384, -0.03875996172428131, 0.4484710693359375, 0.23474973440170288, 0.7751528024673462, 0.08825092762708664, -0.00183810293674469, -0.12937825918197632, -0.15973830223083496, -0.5639431476593018, 0.14194908738136292, 0.6260278224945068, 0.4450017213821411, 0.01620279997587204, -0.31759801506996155, 0.08452150225639343, 0.15003861486911774, -0.4442146420478821, -0.2723245322704315, -0.5077311396598816, 0.292941689491272, -0.17899411916732788, -0.4819982051849365, 0.5620265007019043, 0.5478734374046326, 0.3978990316390991, 0.46397194266319275, 0.7629085779190063, 0.6069497466087341, 0.8380173444747925, -0.00721057690680027, 0.5347946882247925, 0.027407489717006683, -0.20206129550933838, 0.4763733744621277, 0.5516512989997864, 0.029902396723628044, -0.10676416009664536, -0.850759744644165, -0.49473077058792114, 0.5585196018218994, 0.6907113194465637, 0.8357075452804565, 0.3376927971839905, -0.5069529414176941, -0.6062127351760864, 0.09289576858282089, -0.5748048424720764, 0.37862297892570496, -0.24899283051490784, -0.596381425857544, 0.40012162923812866, 0.36203843355178833, 0.4057576358318329, -0.36571037769317627, -0.09966893494129181, -0.6630144119262695, -0.12734000384807587, 0.5435581207275391, 0.6447133421897888, -0.32424700260162354, 0.2092876136302948, 0.016820721328258514, 0.4327358901500702, 0.39795809984207153, -0.12236665934324265, 0.7261624336242676, -0.12678876519203186, 0.5825026631355286, 0.5680538415908813, 0.8354955911636353, -0.21570661664009094, -0.5753690600395203, -0.06445048749446869, 0.762269139289856, 0.040958940982818604, 0.610613226890564, -0.020369894802570343, 0.545011043548584, 0.24279725551605225, -0.5467541217803955, -0.11220651865005493, -0.3527762293815613, -0.1632116287946701, 0.44844406843185425, 0.0735330656170845, -0.5992085337638855, 0.38596898317337036, 0.6763473749160767, 0.6233633160591125, 0.7277296185493469, -0.08953738957643509, -0.5223495960235596, 0.0438774898648262, 0.5235397815704346, 0.7374474406242371, 0.13682124018669128, 0.38984552025794983, 0.501053512096405, 0.544288158416748, -0.10049491375684738, -0.424625426530838, -0.1676091104745865, -0.3632720708847046, 0.44425782561302185, -0.11029613763093948, 0.267566978931427, -0.5106238126754761, 0.7288517355918884, -0.42102405428886414, -0.13694581389427185, 0.6026482582092285, 0.13817527890205383, 0.13015775382518768, 0.6762617230415344, -0.10029622912406921, 0.20783045887947083, -0.162192702293396, -0.5715113878250122, 0.30759158730506897, 0.48154374957084656, 0.0985642671585083, 0.007634159177541733, -0.1648276448249817, -0.16258582472801208, -0.07278992980718613, 0.4513570964336395, 0.5931440591812134, -0.5914018154144287, 0.5612777471542358, -0.2250228375196457, -0.08681446313858032, 0.830161452293396, 0.29748642444610596, 0.5603421926498413, 0.06424661725759506, 0.47141072154045105, 0.5927131772041321, -0.6674312949180603, 0.7419288158416748, -0.29513993859291077, 0.31459540128707886, 0.5818383693695068, 0.32659679651260376, 0.268392950296402, -0.28568950295448303, -0.018027888610959053, 0.43304747343063354, 0.46631941199302673, -0.27536827325820923, -0.04585590213537216, -0.6916893720626831, 0.6683887243270874, -0.1126030832529068, 0.7330220937728882, 0.03757964074611664, 0.6858601570129395, -0.6135091781616211, 0.6542232632637024, 0.4398217797279358, 0.492404967546463, -0.06647583842277527, 0.16967257857322693, -0.0017482973635196686, 0.20925366878509521, -0.7695492506027222, 0.4057987332344055, 0.34007203578948975, 0.2162136435508728, -0.671826183795929, -0.7336825132369995, -0.6024703979492188, 0.04092667996883392, 0.7215957641601562, 0.2598585784435272, -0.21276652812957764, -0.6609749794006348, 0.016991227865219116, -0.19597128033638, 0.3823133111000061, 0.2225387543439865, 0.005574949085712433, -0.1981125921010971, -0.5707398653030396, 0.7470301389694214, -0.3544303774833679, -0.3285144865512848, -0.14890119433403015, -0.45134350657463074, -0.0965925008058548, 0.3089819550514221, -0.17884494364261627, -0.6166671514511108, 0.12891215085983276, -0.39214199781417847, 0.14261552691459656, 0.37430211901664734, -0.4456213116645813, -0.2182871699333191, 0.15843327343463898, 0.10964956879615784, 0.5076656937599182, 0.3321247100830078, -0.3566386103630066, 0.42091280221939087, 0.5414720177650452, -0.0031484663486480713, 0.1866927295923233, -0.5755487680435181, 0.04663671553134918, -0.7046530246734619, 0.033724188804626465, 0.6927210092544556, -0.1401757150888443, 0.43932539224624634, -0.41722944378852844, 0.10006671398878098, -0.6547561883926392, 0.46282902359962463, 0.5828143954277039, -0.7334388494491577, -0.13913242518901825, -0.6098092794418335, -0.6479136943817139, 0.4245348274707794, 0.3338037133216858, -0.1635284274816513, 0.4729422330856323, 0.5108768343925476, -0.022011660039424896, 0.024665050208568573, -0.05691790580749512, -0.218398779630661, -0.3966186046600342, 0.1583661139011383, -0.03012644127011299, -0.06357505917549133, -0.2921290397644043, 0.33946841955184937, -0.34734880924224854, -0.4378834664821625, -0.12415216863155365, -0.09998051822185516, -0.06867769360542297, -0.6437168121337891, -0.20982132852077484, 0.30834880471229553, 0.2875445485115051, 0.4372519552707672, 0.11909030377864838, -0.1766430288553238, 0.7008663415908813, -0.24729323387145996, 0.594479501247406, 0.394461065530777, 0.117470383644104, 0.4977377951145172, 0.3629370629787445, -0.017324209213256836, 0.3771270215511322, 0.17241141200065613, 0.46010226011276245, 0.31846141815185547, 0.8258395195007324, 0.3938941955566406, 0.17513790726661682, 0.6269736289978027, -0.40003448724746704, 0.8995493650436401, -0.13408163189888, 0.8724181056022644, -0.35979482531547546, -0.37670034170150757, -0.09866156429052353, 0.694605827331543, -0.0470467209815979, -0.02683994174003601, -0.5049036145210266, 0.26742756366729736, -0.6264607906341553, 0.796922504901886, 0.535040020942688, 0.5001838207244873, 0.47886133193969727, -0.45671945810317993, 0.7460479736328125, -0.3861486613750458, -0.3984943628311157, 0.5926499366760254, -0.07928475737571716, -0.45993947982788086, 0.5645924806594849, 0.14659500122070312, -0.6987922787666321, -0.04200911521911621, 0.1731117069721222, 0.36910560727119446, 0.5085158944129944, 0.6176918148994446, 0.431500107049942, 0.6580010056495667, -0.016697391867637634, 0.41479015350341797, 0.36367571353912354, -0.39496010541915894, 0.0756823867559433, 0.33099865913391113, -0.17381733655929565, 0.33310019969940186, 0.48350071907043457, -0.46167126297950745, -0.3816671669483185, -0.555742621421814, -0.23208746314048767, -0.06034456193447113, -0.2026902139186859, 0.10303796082735062, -0.6204488277435303, 0.05814579874277115, 0.2336493730545044, -0.25964799523353577, -0.14435985684394836, -0.3136261999607086, -0.23123568296432495, 0.33155521750450134, 0.095643050968647, -0.29650551080703735, 0.7516855597496033, 0.6618789434432983, -0.060329288244247437, -0.35946327447891235, 0.6081808805465698, -0.25912728905677795, -0.0016476213932037354, 0.17137601971626282, 0.6392675042152405, -0.4352356791496277, 0.1956992745399475, -0.5193449854850769, 0.5225331783294678, 0.17254525423049927, 0.1312435120344162, -0.29761940240859985, -0.32733532786369324, 0.48585665225982666, -0.33483290672302246, -0.0964479148387909, 0.8888242244720459, 0.4985889792442322, -0.21826708316802979, 0.19449201226234436, -0.436215341091156, 0.201776385307312, 0.44851216673851013, -0.6555171012878418, 0.5347428321838379, -0.2909778356552124, -0.5662840604782104, -0.13881173729896545, 0.6548766493797302, 0.10902091860771179, 0.7961196899414062, 0.36240535974502563, 0.5649623870849609, 0.10712428390979767, 0.2920539975166321, 0.6593831181526184, -0.2834222614765167, 0.10712209343910217, -0.17622041702270508, 0.04392886161804199, 0.2236671894788742, 0.7505307793617249, -0.27446919679641724, -0.6810324192047119, 0.1548995077610016, -0.4247360825538635, -0.5609066486358643, -0.20674265921115875, 0.11560453474521637, 0.3207583427429199, 0.06738309562206268, -0.5062588453292847, 0.4307422637939453, -0.3465149998664856, 0.3240157961845398, -0.3002604842185974, 0.8831269145011902, -0.3617230951786041, 0.5076888799667358, -0.5335904955863953, -0.04292255640029907, 0.8637173175811768, -0.42449161410331726, -0.19949711859226227, 0.2854223847389221, 0.5285306572914124, 0.34136247634887695, -0.01601482555270195, 0.6528439521789551, 0.6696815490722656, 0.5665247440338135, 0.000599157065153122, -0.5904613733291626, 0.19373434782028198, 0.7905173897743225, -0.3212890923023224, 0.4665001630783081, -0.5652803182601929, 0.660376787185669, -0.18631815910339355, 0.1503039449453354, -0.1264604777097702, 0.3098442554473877, 0.46254435181617737, -0.15502113103866577, -0.19138103723526, 0.09914403408765793, 0.23202095925807953, 0.39408546686172485, 0.23550362884998322, -0.26632264256477356, -0.7071288824081421, -0.09809648245573044, -0.05866057425737381, -0.20874933898448944, 0.5648868680000305, 0.048399507999420166, 0.09275034815073013, -0.11833646893501282, -0.6833159923553467, -0.4832107424736023, 0.7181835770606995, 0.44653841853141785, 0.3513978123664856, -0.28749895095825195, 0.25456100702285767, 0.2849273085594177, 0.24087291955947876, 0.5260958671569824, 0.6827409863471985, 0.10052025318145752, -0.06596437841653824, -0.6324977874755859, 0.10594986379146576, -0.3409154415130615, -0.08146344125270844, 0.13492663204669952, 0.478783518075943, 0.4134276211261749, 0.5201202034950256, -0.4331982433795929, -0.48481279611587524, 0.0002752300351858139, 0.1281784474849701, 0.46551379561424255, -0.29144158959388733, -0.5817666053771973, 0.0861540138721466, -0.2814759612083435, -0.5523887872695923, 0.1741785705089569, 0.41274771094322205, 0.2524617314338684, 0.11956916004419327, -0.327175498008728, 0.09381720423698425, 0.18635490536689758, 0.6965212821960449, -0.17190569639205933, 0.5062975883483887, 0.6928202509880066, 0.33384189009666443, 0.16385185718536377, 0.20876261591911316, -0.8773955702781677, -0.19910188019275665, -0.5044702291488647, -0.06425918638706207, 0.08282840251922607, 0.4336091876029968, 0.3851351737976074, 0.2467946708202362, -0.08031680434942245, 0.2465677410364151, 0.542986273765564, -0.576008677482605, 0.13706986606121063, 0.1509503722190857, 0.7289201021194458, 0.6593160033226013, -0.143152117729187, 0.20680050551891327, 0.36873096227645874, 0.12132684886455536, -0.5812473893165588, 0.653080940246582, 0.12694579362869263, -0.5581386685371399, 0.32003387808799744, 0.044709622859954834, 0.8721939325332642, 0.11913320422172546, 0.17411118745803833, 0.7911044359207153, 0.225993350148201, 0.633725106716156, 0.6731399297714233, -0.45893606543540955, 0.32357537746429443, -0.21888437867164612, 0.7222169637680054, 0.043982863426208496, 0.5044512748718262, 0.020029954612255096, 0.14469707012176514, 0.4857819676399231, 0.7432364225387573, 0.39677125215530396, 0.6910150051116943, 0.08142629265785217, 0.3205183744430542, 0.5293572545051575, -0.061872802674770355, 0.2411869913339615, 0.4518905282020569, -0.045160867273807526, -0.010531164705753326, -0.13681840896606445, -0.20003125071525574, 0.04870378598570824, 0.16014620661735535, -0.4495147466659546, 0.7294266819953918, 0.7839823961257935, 0.7861634492874146, -0.47479504346847534, 0.08800670504570007, 0.6312564611434937, 0.5851925611495972, -0.7567087411880493, 0.3785811960697174, 0.14516380429267883, 0.009355634450912476, 0.010739978402853012, -0.7449877262115479, -0.5468195676803589, -0.26689276099205017, 0.05047938972711563, 0.3014569878578186, 0.5671384334564209, 0.23209261894226074, -0.6205209493637085, -0.6888680458068848, 0.13125264644622803, 0.457168847322464, -0.6451299786567688, -0.1873749941587448, 0.34581077098846436, 0.09038595855236053, 0.7727934122085571, 0.17761875689029694, -0.2399870604276657, 0.03985375165939331, 0.10663782060146332, 0.6963061690330505, -0.2427583783864975, 0.6452996730804443, -0.30917757749557495, 0.37492847442626953, -0.3135972023010254, 0.1393708735704422, -0.7276909351348877, 0.11216975003480911, -0.4428735375404358, -0.5585198998451233, 0.7458577156066895, 0.18926411867141724, 0.4073254466056824, -0.5767701864242554, -0.07107877731323242, 0.1909031718969345, 0.22540582716464996, 0.6339468955993652, -0.15307262539863586, -0.2188165783882141, 0.49879369139671326, -0.4000324308872223, -0.6342724561691284, -0.13980358839035034, -0.7265973687171936, 0.06896263360977173, 0.0189303457736969, 0.2240508794784546, -0.2872256338596344, 0.8099315762519836, -0.4396865665912628, 0.7771812677383423, 0.2882425785064697, 0.2717665731906891, -0.38802823424339294, 0.6785358190536499, -0.5424537658691406, 0.5652846693992615, -0.06779778003692627, -0.6647745370864868, -0.4677829146385193, -0.5669140815734863, 0.51554936170578, 0.48694339394569397, 0.3857886791229248, 0.2117844820022583, -0.42900627851486206, 0.5077345371246338, -0.26549363136291504, -0.4697626829147339, 0.6474510431289673, 0.666093647480011, -0.8605548143386841, 0.5333503484725952, -0.18487021327018738, -0.7184882164001465, 0.46744200587272644, -0.5767962336540222, -0.05347162485122681, -0.21722093224525452, 0.34223467111587524, -0.4365044832229614, 0.3489752411842346, -0.549564778804779, 0.4096054434776306, -0.27255669236183167, 0.3562660217285156, 0.473387748003006, -0.5221951007843018, -0.4474014937877655, -0.017945438623428345, 0.4865927994251251, 0.7982234954833984, 0.4213724732398987, -0.13284453749656677, 0.5493001937866211, -0.17471083998680115, 0.3813995122909546, 0.25574296712875366, 0.21563664078712463, -0.33500856161117554, -0.6679137945175171, 0.30149978399276733, 0.2763212323188782, -0.3562568426132202, -0.5175012946128845, -0.37513989210128784, 0.7268239259719849, 0.5682178735733032, -0.7546896934509277, 0.40612316131591797, 0.08031975477933884, -0.09403979778289795, -0.7744797468185425, -0.7622443437576294, 0.024530665948987007, 0.06469045579433441, 0.0977139100432396, 0.5705983638763428, 0.15562376379966736, -0.2975997030735016, 0.25273293256759644, -0.4577331840991974, 0.06863538175821304, 0.10321727395057678, 0.8030970096588135, 0.12664934992790222, 0.5413007140159607, -0.3706977665424347, -0.6026647090911865, 0.658598005771637, -0.3457251787185669, -0.04999656230211258, 0.18013209104537964, -0.0840609222650528, -0.0453774631023407, -0.2781245708465576, 0.266904354095459, 0.3361664414405823, 0.5804433822631836, -0.17622219026088715, 0.10206755250692368, -0.04516325891017914, 0.5400106906890869, -0.2442476451396942, 0.5359169244766235, 0.6395742893218994, 0.06003114581108093, 0.24074652791023254, 0.6702216267585754, -0.7362715005874634, 0.12211039662361145, 0.5978507995605469, -0.3360138535499573, 0.07017232477664948, -0.1798633337020874, 0.7225490212440491, 0.8234115242958069, 0.3411600887775421, 0.5445746183395386, -0.2493283599615097, -0.3758876919746399, -0.6240676045417786, -0.02143290638923645, -0.5765277147293091, 0.3210941553115845, -0.35969042778015137, 0.09672985970973969, 0.03439325839281082, 0.1569453328847885, 0.7858011722564697, 0.7225660085678101, -0.8332672119140625, 0.5493295192718506, 0.40051865577697754, -0.15554796159267426, -0.009880580008029938, -0.1788141429424286, -0.04944133758544922, 0.5227091312408447, 0.5867958068847656, 0.3664294183254242, -0.2604169249534607, 0.06193753331899643, -0.18655520677566528, 0.058782413601875305, 0.27639028429985046, -0.18311455845832825, 0.21502406895160675, 0.06259464472532272, -0.18557408452033997, -0.21981139481067657, -0.7264014482498169, 0.34806346893310547, -0.001712348312139511, -0.39238280057907104, -0.4855436086654663, 0.5236097574234009, 0.25925374031066895, 0.4666345715522766, 0.35970237851142883, -0.40431779623031616, 0.31843799352645874, 0.7832298278808594, -0.2632508873939514, 0.01028895378112793, 0.027375921607017517, -0.15454724431037903, 0.3867053687572479, 0.49454474449157715, -0.1487588733434677, 0.5064623355865479, -0.25999927520751953, -0.3277010917663574, -0.4306681156158447, -0.829020619392395, 0.0444570928812027, 0.5274654626846313, 0.7242363691329956, 0.33377814292907715, 0.6111951470375061, -0.07159454375505447, -0.2255682349205017, 0.6455498933792114, -0.14970065653324127, 0.5534086227416992, -0.5683711767196655, 0.13130146265029907, -0.16731885075569153, 0.09163888543844223, 0.0028263404965400696, -0.23621194064617157, -0.5355157852172852, -0.7454063892364502, -0.06963729858398438, 0.5993707776069641, -0.3158721923828125, -0.39497965574264526, -0.06637408584356308, -0.07115089893341064, 0.4520864188671112, -0.07565926015377045, -0.8899571299552917, -0.6114871501922607, 0.24952660501003265, 0.02447120100259781, -0.02742079645395279, 0.552161455154419, 0.44890880584716797, -0.6865647435188293, 0.039799466729164124, -0.6961499452590942, 0.40582403540611267, 0.5748034715652466, 0.17772901058197021, 0.2043975442647934, 0.14608339965343475, 0.10134381800889969, -0.06701125204563141, 0.6991232633590698, -0.20620015263557434, 0.45053425431251526, 0.24541118741035461, -0.405677855014801, 0.19950853288173676, -0.20323818922042847, 0.5689551830291748, -0.06955737620592117, 0.5200532674789429, -0.3528812825679779, 0.392480731010437, -0.15076632797718048, 0.059220924973487854, 0.5944859981536865, -0.5730260610580444, 0.7420868873596191, -0.3000434637069702, -0.14585226774215698, -0.3324522376060486, -0.38025790452957153, 0.315817654132843, 0.09281982481479645, 0.37259796261787415, 0.19643577933311462, -0.22082072496414185, -0.6804059743881226, 0.31557372212409973, 0.7407896518707275, -0.4380972385406494, 0.5602895021438599, 0.00912042148411274, 0.31135794520378113, 0.5396666526794434, -0.2776820957660675, 0.04214991629123688, 0.6161277294158936, 0.20247091352939606, 0.033753059804439545, 0.0012477487325668335, -0.17457979917526245, 0.47356826066970825, 0.19125311076641083, 0.04537326097488403, -0.37003907561302185, -0.5832580327987671, 0.2004534751176834, 0.7143844962120056, 0.35171955823898315, 0.37641972303390503, 0.2529163956642151, -0.22240711748600006, 0.36772358417510986, 0.062058065086603165, 0.5543117523193359, -0.03242474049329758, 0.1908150315284729, 0.6060473918914795, 0.05422307550907135, 0.6574081182479858, 0.7722002863883972, -0.20157036185264587, -0.3048211336135864, -0.290092408657074, 0.7149933576583862, -0.11738002300262451, -0.1420646458864212, -0.07236625254154205, -0.6665419340133667, 0.31165874004364014, -0.5809059143066406, -0.5117689371109009, 0.2892550826072693, 0.44947269558906555, 0.7440187335014343, 0.2640857696533203, 0.33483320474624634, 0.6593541502952576, -0.08031722158193588, 0.16637113690376282, -0.038984768092632294, 0.29996728897094727, -0.2887150049209595, 0.26792240142822266, 0.29579615592956543, -0.33439862728118896, -0.03776925802230835, -0.32972633838653564, 0.35628122091293335, 0.22525566816329956, 0.6383370161056519, 0.8035828471183777, 0.687955915927887, -0.6300753355026245, 0.19897706806659698, 0.6765739321708679, 0.17750540375709534, 0.3414613604545593, 0.49387043714523315, 0.28993192315101624, 0.20815351605415344, -0.49620622396469116, -0.7017310857772827, 0.24235206842422485, 0.37597593665122986, 0.06324982643127441, 0.4101049602031708, 0.032954804599285126, -0.4688310921192169, -0.33461594581604004, -0.22272463142871857, -0.36607444286346436, 0.5505937933921814, 0.7556650638580322, 0.5759901404380798, 0.24337199330329895, -0.4738158881664276, 0.11060144007205963, -0.38766324520111084, 0.3293595314025879, -0.36333227157592773, -0.43182945251464844, 0.23957131803035736, -0.23623718321323395, 0.7248977422714233, -0.40141552686691284, 0.8118157386779785, -0.593839168548584, -0.4528861343860626, 0.10224029421806335, 0.16525623202323914, -0.8462625741958618, -0.24591365456581116, -0.020510271191596985, -0.2231166958808899, -0.8467490673065186, -0.27993929386138916, 0.038637399673461914, 0.11262750625610352, 0.14839281141757965, 0.4534817337989807, -0.018441468477249146, -0.05381963774561882, -0.12896807491779327, 0.34645912051200867, 0.4246109127998352, 0.40198373794555664, 0.10160006582736969, 0.46085670590400696, 0.46670880913734436, 0.5677038431167603, -0.263216108083725, 0.029583781957626343, -0.4581011235713959, 0.49356454610824585, 0.6390454769134521, 0.2273722141981125, 0.5872083902359009, -0.5060101747512817, -0.05998988449573517, 0.4201831519603729, 0.4481068253517151, 0.3848072290420532, 0.6950927972793579, 0.13974115252494812, -0.7910636067390442, -0.399128258228302, -0.1253114938735962, 0.5627232789993286, -0.2815091013908386, -0.44952529668807983, -0.6882904171943665, 0.53367018699646, 0.22894011437892914, 0.3967767059803009, 0.5258102416992188, 0.17899887263774872, -0.5055789947509766, -0.313599556684494, 0.5578537583351135, -0.30052727460861206, -0.34010496735572815, -0.4514661729335785, 0.3984411358833313, 0.4341098666191101, 0.23705598711967468, 0.17822739481925964, 0.16862772405147552, -0.007784925401210785, 0.4620431065559387, 0.32129228115081787, 0.039427340030670166, 0.031049853190779686, 0.17064417898654938, 0.06319534778594971, -0.12475882470607758, 0.6081836223602295, 0.08197236061096191, -0.43950051069259644, 0.43272078037261963, -0.7258301973342896, 0.49032068252563477, 0.5369265675544739, 0.5240930914878845, 0.7495987415313721, 0.2702736258506775, 0.6664864420890808, 0.041908085346221924, 0.7424660325050354, 0.3371303975582123, 0.28911012411117554, 0.11757181584835052, 0.29332661628723145, -0.28549572825431824, 0.524901270866394, -0.06222693622112274, -0.4986919164657593, 0.29418736696243286, 0.5732032060623169, 0.6253304481506348, 0.6873472332954407, -0.210577130317688, 0.06013525277376175, 0.5174997448921204, -0.055923961102962494, 0.3681983947753906, 0.3325490355491638, 0.1718655526638031, 0.7057899236679077, 0.09787951409816742, -0.33093777298927307, 0.40564846992492676, 0.5267874002456665, -0.3597602844238281, 0.5450727939605713, 0.37893763184547424, 0.11418429017066956, 0.6597416996955872, -0.48034045100212097, 0.04321570694446564, -0.5168246030807495, -0.4612206220626831, -0.05691627785563469, 0.31113553047180176, 0.29868555068969727, -0.7987220287322998, -0.04962952435016632, 0.02501346915960312, 0.6915018558502197, 0.8373112678527832, 0.1482841670513153, 0.08790013939142227, -0.23782899975776672, 0.018463507294654846, -0.7978489995002747, -0.6623491048812866, 0.057508960366249084, -0.025166872888803482, 0.22316524386405945, -0.20248481631278992, 0.15954062342643738, -0.1847037523984909, 0.24837833642959595, -0.4960702955722809, 0.5627461671829224, 0.3129847049713135, -0.14929647743701935, -0.1446734368801117, 0.234143927693367, 0.24048221111297607, 0.5847978591918945, -0.46988821029663086, 0.15237018465995789, 0.45072436332702637, -0.5892539024353027, 0.6019142866134644, 0.42980509996414185, 0.5138275623321533, -0.43953874707221985, -0.2604963481426239, -0.725327730178833, -0.5871783494949341, 0.4136829674243927, -0.2946432828903198, -0.38884323835372925, -0.5905486941337585, 0.3426437973976135, 0.271243155002594, -0.38558125495910645, 0.4270343780517578, 0.2134636640548706, 0.012266606092453003, 0.16011770069599152, 0.650649905204773, -0.5125860571861267, -0.19166496396064758, 0.45061784982681274, 0.5379013419151306, -0.5586076974868774, -0.5211794376373291, 0.8717939257621765, 0.011095985770225525, -0.7965312600135803, 0.4064295291900635, -0.6375466585159302, 0.5897133946418762, 0.765537440776825, 0.3421533703804016, -0.4939362406730652, 0.601026177406311, 0.0804271399974823, 0.3674744665622711, 0.5062659978866577, 0.2810981869697571, -0.2981826961040497, -0.5576473474502563, 0.5251035690307617, -0.2599998414516449, -0.02022811770439148, 0.3431546688079834, 0.7149831056594849, 0.13355350494384766, -0.4220010042190552, 0.6999433636665344, -0.037425994873046875, -0.1781509816646576, -0.6693922877311707, 0.18946003913879395, -0.8165615797042847, 0.5204453468322754, 0.2702614665031433, 0.587396502494812, 0.17927046120166779, 0.46223682165145874, 0.5699029564857483, -0.5706236362457275, 0.31598520278930664, 0.4616544842720032, 0.7537155747413635, 0.7245878577232361, 0.7324498295783997, -0.46927255392074585, 0.5220043659210205, 0.4324452877044678, 0.7524120807647705, -0.05316130816936493, 0.16323205828666687, 0.3583199977874756, -0.3509085774421692, -0.5108881592750549, -0.4327820837497711, 0.3222639560699463, 0.042492859065532684, -0.4026091396808624, 0.42782288789749146, -0.5432929992675781, -0.04339958727359772, 0.49019545316696167, 0.5273724794387817, -0.4114643633365631, -0.17122501134872437, 0.08012285828590393, 0.6367290019989014, -0.466886430978775, -0.23985691368579865, -0.2750406563282013, -0.368964821100235, 0.34383466839790344, -0.035222284495830536, -0.006170481443405151, 0.09335623681545258, 0.6151070594787598, 0.637892484664917, -0.44912171363830566, 0.06787969172000885, 0.6021071076393127, 0.23086582124233246, -0.13722114264965057, 0.20141875743865967, 0.31462809443473816, 0.36153021454811096, -0.29004397988319397, 0.1236054077744484, -0.18757836520671844, 0.024203866720199585, -0.5231649875640869, -0.19334810972213745, -0.025738738477230072, -0.39228466153144836, -0.11488431692123413, -0.4914674162864685, 0.17378278076648712, -0.31377118825912476, 0.19033542275428772, -0.6436587572097778, 0.2615576982498169, 0.03773465007543564, 0.7986739873886108, 0.16336870193481445, 0.7006585001945496, -0.09781524538993835, 0.0007008165121078491, 0.5171137452125549, 0.6557670831680298, -0.05430655926465988, -0.3899862766265869, 0.5177592039108276, 0.5766915082931519, 0.7188403606414795, -0.19542019069194794, -0.1224418431520462, 0.10759269446134567, -0.1400071084499359, -0.19233793020248413, -0.5093274116516113, 0.10380598902702332, -0.4842824637889862, -0.5611615777015686, -0.3534647226333618, 0.5817846655845642, -0.11327344924211502, 0.6822199821472168, 0.07238218188285828, -0.2663831114768982, 0.6755131483078003, 0.44303303956985474, -0.30481693148612976, -0.6252736449241638, -0.0532340481877327, 0.2946787476539612, 0.5504366159439087, 0.5634931325912476, -0.18865498900413513, -0.12568527460098267, -0.15724052488803864, 0.5588160753250122, 0.043836355209350586, 0.7461702227592468, -0.11813509464263916, 0.8331042528152466, -0.488764226436615, 0.4436696171760559, 0.34129321575164795, 0.43367451429367065, 0.5863733291625977, -0.6611418724060059, 0.6573538780212402, 0.6514437198638916, -0.18519258499145508, 0.07548639178276062, -0.4149929881095886, -0.019640054553747177, 0.2951098680496216, 0.09429218620061874, 0.08331593871116638, 0.41276663541793823, 0.6289480924606323, -0.30056503415107727, -0.5946412682533264, 0.3793235421180725, -0.3738275170326233, 0.42583075165748596, 0.18702173233032227, 0.06562002748250961, 0.2555658221244812, 0.5320516228675842, -0.21994104981422424, 0.328594833612442, 0.5973195433616638, -0.6132410168647766, 0.10712407529354095, -0.8133282661437988]


In [27]:
sum(cosine_similarities_random_graphs) / len(cosine_similarities_random_graphs)

0.08385531850513958